#Elaborato Luigi Pio Faletra su MLP

##Funzioni

In [ ]:
def images_reader(directory):
    """
    Esegue il caricamento delle immagini della cartella 'faces', convertendole in scala di grigi, ridimensionandole, ruotandole e filtrandole.
    Carica ogni immagine ridimensionata, ruotata e filtrata nella lista delle immagini, alla sottolista contenente immagini aventi stesse dimensioni. 
    La prima sottolista ha le immagini di dimensioni 30x30, la seconda sottolista quelle di dimensioni 60x60 e la terza sottolista quelle di dimensioni 120x120.
    Richiama la funzione 'images_labelling' per ottenere le label per la direzione e l'emozione del volto.
    """

    direction, emotion, image = [[], [], []], [[], [], []], [[], [], []]
    directory_list = ['an2i_png', 'at33_png', 'boland_png', 'bpm_png',
                      'ch4f_png', 'cheyer_png', 'choon_png', 'danieln_png',
                      'glickman_png', 'karyadi_png', 'kawamura_png', 'kk49_png', 
                      'megak_png', 'mitchell_png', 'night_png', 'phoebe_png', 
                      'saavik_png', 'steffi_png', 'sz24_png', 'tammo_png']
    print("Caricamento immagini in corso...")

    for i in range(0, len(directory_list)):
        subdirectory = directory + directory_list[i]
        print("\nSottocartella:", subdirectory)

        for filename in listdir(subdirectory):
            print(filename)
            f_name, f_extension = splitext(filename)
            img = Image.open(subdirectory + '/' + filename)
            i = 0
            img = img.convert('1')

            if   img.size[0] == 32: i = 0
            elif img.size[0] == 64: i = 1
            elif img.size[0] == 128: i = 2
            
            if   i == 0: img = img.resize((30, 30))
            elif i == 1: img = img.resize((60, 60))
            elif i == 2: img = img.resize((120, 120))
            
            seed()
            angle = randint(0, 360)
            rotated_image = img.rotate(angle, fillcolor='white')
            filtered_image = img.filter(ImageFilter.SHARPEN)
            image[i].append(img)
            image[i].append(rotated_image)
            image[i].append(filtered_image)
            index = f_name.split('_')
            images_labelling(f_name, direction, i, 'direzione')
            images_labelling(f_name, emotion, i, 'emozione')
    print("\nLabel per le direzioni:", direction)
    print("Label per le emozioni:", emotion)
    return direction, emotion, image

def images_labelling(name, label_array, i, array_type):
    """
    Ottiene delle label in base ai nomi dei file nel dataset, da inserire in due liste separate, denominate 'direction' ed 'emotion'.
    Per la classificazione della direzione del volto, sono ricercate le parole:
        - left, a cui si assegna la label 0;
        - right, a cui viene associata la label 1;
        - straight, a cui corrisponde la label 2;
        - up, a cui è collegata la label 3.
    Per la classificazione dell'emozione del volto, sono ricercate le parole:
        - happy, a cui si assegna la label 0;
        - angry, a cui viene associata la label 1;
        - neutral, a cui corrisponde la label 2;
        - sad, a cui è collegata la label 3.
    """

    if array_type == 'direzione':
        if "left" in name:
            for j in range(3):
                label_array[i].append(0)
        elif "right" in name:
            for j in range(3):
                label_array[i].append(1)
        elif "straight" in name:
            for j in range(3):
                label_array[i].append(2)
        elif "up" in name:
            for j in range(3):
                label_array[i].append(3)

    elif array_type == 'emozione':
        if "angry" in name:
            for j in range(3):
                label_array[i].append(0)
        elif "happy" in name:
            for j in range(3):
                label_array[i].append(1)
        elif "neutral" in name:
            for j in range(3):
                label_array[i].append(2)
        elif "sad" in name:
            for j in range(3):
                label_array[i].append(3)

def array_creation_for_label(label_array):
    """
    Trasforma ogni sottolista della lista di label in un array e mostra il contenuto di tali array.
    """

    array30 = np.array(label_array[0])
    array60 = np.array(label_array[1])
    array120 = np.array(label_array[2])
    print(array30, array60, array120, sep='\n\n', end='\n\n')
    return array30, array60, array120

def first_array_creation_for_images(image):
    """
    Trasforma ogni sottolista di immagini in un array con valori dipendenti dall'immagine elaborata, immessi in una lista, e mostra il contenuto di tali array.
    """

    total_image = [[], [], []]

    for i in range(3):
        for img in image[i]:
          val = img.getdata()
          single_img = np.array(val)
          total_image[i].append(single_img)
    print(np.array(total_image[0]), np.array(total_image[1]), np.array(total_image[2]), sep='\n\n')
    return total_image

def second_array_creation_for_images(total_image):
    """
    Trasforma ogni array della lista creata con la funzione 'first_array_creation_for_images' in un array con valori binari e mostra il contenuto di tali array.
    """

    tot_image = [[], [], []]

    for i in range(3):
        tot_image[i] = np.array(total_image[i])
        tot_image[i] //= 255
        print(tot_image[i])
    return tot_image

def first_reshape_and_conversion(tot_image, direction30, direction60, direction120, emotion30, emotion60, emotion120):
    """
    Effettua il reshape degli array della lista ottenuta dalla funzione 'second_array_creation_for_images' e converte gli array di interi 
    della lista ottenuta dalla funzione 'array_creation_for_label' in delle matrici binarie, mostrando i risultati di tali operazioni.
    """

    label_list = [direction30, direction60, direction120, emotion30, emotion60, emotion120]
    dimensions_tuple = (30, 60, 120)
    print("\nPrima del reshape")

    for i in range(3): 
        print(tot_image[i].shape)
        tot_image[i] = tot_image[i].reshape(tot_image[i].shape[0], dimensions_tuple[i], dimensions_tuple[i], 1)
    print("Dopo il reshape")

    for i in range(3):
        print(tot_image[i].shape)
    print("Conversione")

    for i in range(6):
        label_list[i] = to_categorical(label_list[i], 4)
        print(label_list[i].shape)
    return tot_image, label_list[0], label_list[1], label_list[2], label_list[3], label_list[4], label_list[5]

def image_definition(image):
    """
    Assegna a tre variabili diverse i tre array ottenuti dalla funzione 'second_array_creation_for_images'.
    """

    image30 = image[0]
    image60 = image[1]
    image120 = image[2]
    return image30, image60, image120

def train_and_test_creation(tot_image, label_array, dimension, type_label):
    """
    Effettua la divisione degli array in insiemi di train e test, mostrando i relativi shape. 
    Il parametro 'shuffle' di 'train_test_split' è impostato come attivo di default.
    """

    x_train, x_test, y_train, y_test = train_test_split(tot_image, label_array, test_size=0.3)
    print("Shape x_train", dimension, " per ", type_label, ": ", x_train.shape, sep='')
    print("Shape x_test", dimension, " per ", type_label, ": ", x_test.shape, sep='')
    print("Shape y_train", dimension, " per ", type_label, ": ", y_train.shape, sep='')
    print("Shape y_test", dimension, " per ", type_label, ": ", y_test.shape, sep='')
    return x_train, x_test, y_train, y_test

def train_and_test_reshape(x_train30, x_test30, x_train60, x_test60, x_train120, x_test120, type_label):
    """
    Effettua il reshape della variabili di train e test ottenute dalla funzione 'train_and_test_creation', mostrando i relativi shape prima e dopo il reshape.
    """

    train_and_test_list = [x_train30, x_test30, x_train60, x_test60, x_train120, x_test120]
    print("Prima del reshape per", type_label)

    for i in range(6):
        print(train_and_test_list[i].shape)
        train_and_test_list[i] = train_and_test_list[i].reshape(train_and_test_list[i].shape[0], train_and_test_list[i].shape[1]*train_and_test_list[i].shape[2]*train_and_test_list[i].shape[3])
    print("Dopo il reshape per", type_label)
    
    for i in range(6):
        print(train_and_test_list[i].shape)
    return train_and_test_list[0], train_and_test_list[1], train_and_test_list[2], train_and_test_list[3], train_and_test_list[4], train_and_test_list[5]

def model_fitting(model, x_train, y_train, number_word):
    """
    Effettua il fitting del modello creato con gli insiemi 'x_train' e 'y_train' delle immagini associate a tale modello, mostrandone iterazioni e tasso di loss.
    """

    words_tuple = ('piccole', 'medie', 'grandi')
    print('Immagini', words_tuple[number_word])
    history = model.fit(x_train, y_train)
    return history

def predictions_realizing(model, x_train, x_test):
    """
    Effettua una predizione dei risultati usufruendo degli insiemi 'x_train' e 'x_test' associati al modello passato come parametro.
    """

    p_train = model.predict(x_train)
    p_test = model.predict(x_test)
    return p_train, p_test

def score_history(y_train, p_train, y_test, p_test, number_word):
    """
    Fornisce i risultati di accuracy sugli insiemi 'y_train', 'p_train', 'y_test', 'p_test' associati al modello, mostrandone i risultati per train e test.
    """
    
    words_tuple = ('piccole', 'medie', 'grandi')
    accuracy_train = accuracy_score(y_train, p_train)
    accuracy_test = accuracy_score(y_test, p_test)
    print('Train score modello per immagini ', words_tuple[number_word], ': ', accuracy_train, sep='')
    print('Test score modello per immagini ', words_tuple[number_word], ': ', accuracy_test, sep='')
    return accuracy_train, accuracy_test

##Codice

In [ ]:
# Elenco delle librerie e delle funzioni importate da alcune librerie necessarie per i compiti svolti nel progetto.
from keras.utils.np_utils import to_categorical
from os import listdir
from os.path import splitext
from PIL import Image, ImageFilter
from random import randint, seed
from shutil import unpack_archive
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np

# Cartelle in cui risiede il dataset di immagini, la cui scelta dipende dalla modalità di caricamento delle immagini.
dir1 = '/content/faces/'
dir2 = '/content/drive/MyDrive/faces/'
zip = '/content/faces.zip'

"""
Modalità di caricamento delle immagini:
1 - Tramite Google Drive;
2 - Tramite tasto 'Carica' della sezione 'File' di Google Colab.
"""
upload_mode = 1

# Richiamo della funzione 'images_reader' per la creazione delle liste di immagini e label.
if upload_mode == 1:
    unpack_archive(zip)
    direction, emotion, image = images_reader(dir1)
elif upload_mode == 2:
    from google.colab import drive
    drive.mount('/content/drive')
    direction, emotion, image = images_reader(dir2)

Caricamento immagini in corso...

Sottocartella: /content/faces/an2i_png
an2i_left_neutral_sunglasses.png
an2i_straight_sad_open_4.png
an2i_straight_happy_sunglasses_2.png
an2i_left_neutral_open_4.png
an2i_left_neutral_sunglasses_2.png
an2i_straight_neutral_sunglasses_4.png
an2i_right_happy_sunglasses.png
an2i_right_neutral_open_2.png
an2i_right_angry_sunglasses_4.png
an2i_left_neutral_open.png
an2i_straight_happy_sunglasses.png
an2i_left_sad_sunglasses.png
an2i_left_happy_sunglasses_4.png
an2i_right_happy_open.png
an2i_up_neutral_sunglasses_2.png
an2i_right_happy_open_2.png
an2i_right_sad_sunglasses_4.png
an2i_up_happy_open_4.png
an2i_right_angry_sunglasses.png
an2i_up_happy_sunglasses_2.png
an2i_up_neutral_open_4.png
an2i_left_angry_open_2.png
an2i_up_happy_open.png
an2i_straight_neutral_open_2.png
an2i_straight_angry_open_2.png
an2i_right_sad_sunglasses_2.png
an2i_right_angry_open.png
an2i_up_angry_open.png
an2i_straight_angry_sunglasses_2.png
an2i_up_sad_open.png
an2i_left_angry_su

In [ ]:
# Richiamo della funzione 'array_creation_for_label' per differenziare le label di direzione ed emozione delle immagini in base alla dimensione delle stesse.
direction30, direction60, direction120 = array_creation_for_label(direction)
emotion30, emotion60, emotion120 = array_creation_for_label(emotion)

[2 2 2 ... 3 3 3]

[2 2 2 ... 3 3 3]

[0 0 0 ... 3 3 3]

[3 3 3 ... 2 2 2]

[1 1 1 ... 2 2 2]

[2 2 2 ... 3 3 3]



In [ ]:
# Richiamo della funzione 'first_array_creation_for_images' per creare array dalla lista di immagini.
total_image = first_array_creation_for_images(image)

[[  0   0   0 ... 255   0   0]
 [255 255 255 ... 255 255 255]
 [  0   0   0 ... 255   0   0]
 ...
 [  0   0   0 ... 255   0   0]
 [255 255 255 ... 255 255 255]
 [  0   0   0 ... 255   0   0]]

[[  0   0   0 ...   0   0   0]
 [255 255 255 ... 255 255 255]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [255   0   0 ...   0   0 255]
 [  0   0   0 ...   0   0   0]]

[[  0   0   0 ...   0   0   0]
 [255 255 255 ... 255 255 255]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0 255   0]
 [255 255 255 ... 255 255 255]
 [  0   0   0 ...   0 255   0]]


In [ ]:
"""
Richiamo delle funzioni 'second_array_creation_for_images', 'first_reshape_and_conversion' e 'image_definition' per la creazione di nuovi array, 
il reshape e la conversione di alcuni array e la loro suddivisione in base alle dimensioni delle immagini.
"""
tot_image = second_array_creation_for_images(total_image)

tot_image, direction30, direction60, direction120, emotion30, emotion60, emotion120 = first_reshape_and_conversion(tot_image, direction30, direction60, direction120, emotion30, emotion60, emotion120)

tot_image30, tot_image60, tot_image120 = image_definition(tot_image)

[[0 0 0 ... 1 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 1 0 0]]
[[0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 1 0]]

Prima del reshape
(1872, 900)
(1872, 3600)
(1872, 14400)
Dopo il reshape
(1872, 30, 30, 1)
(1872, 60, 60, 1)
(1872, 120, 120, 1)
Conversione
(1872, 4)
(1872, 4)
(1872, 4)
(1872, 4)
(1872, 4)
(1872, 4)


In [ ]:
# Richiamo della funzione 'train_and_test_creation' per creare insiemi diversificati per dimensione dell'immagine e caratteristica del volto osservata.
x_train30_direction, x_test30_direction, y_train30_direction, y_test30_direction = train_and_test_creation(tot_image30, direction30, '30', 'direzione')
x_train30_emotion, x_test30_emotion, y_train30_emotion, y_test30_emotion = train_and_test_creation(tot_image30, emotion30, '30', 'emozione')

x_train60_direction, x_test60_direction, y_train60_direction, y_test60_direction = train_and_test_creation(tot_image60, direction60, '60', 'direzione')
x_train60_emotion, x_test60_emotion, y_train60_emotion, y_test60_emotion = train_and_test_creation(tot_image60, emotion60, '60', 'emozione')

x_train120_direction, x_test120_direction, y_train120_direction, y_test120_direction = train_and_test_creation(tot_image120, direction120, '120', 'direzione')
x_train120_emotion, x_test120_emotion, y_train120_emotion, y_test120_emotion = train_and_test_creation(tot_image120, emotion120, '120', 'emozione')

Shape x_train30 per direzione: (1310, 30, 30, 1)
Shape x_test30 per direzione: (562, 30, 30, 1)
Shape y_train30 per direzione: (1310, 4)
Shape y_test30 per direzione: (562, 4)
Shape x_train30 per emozione: (1310, 30, 30, 1)
Shape x_test30 per emozione: (562, 30, 30, 1)
Shape y_train30 per emozione: (1310, 4)
Shape y_test30 per emozione: (562, 4)
Shape x_train60 per direzione: (1310, 60, 60, 1)
Shape x_test60 per direzione: (562, 60, 60, 1)
Shape y_train60 per direzione: (1310, 4)
Shape y_test60 per direzione: (562, 4)
Shape x_train60 per emozione: (1310, 60, 60, 1)
Shape x_test60 per emozione: (562, 60, 60, 1)
Shape y_train60 per emozione: (1310, 4)
Shape y_test60 per emozione: (562, 4)
Shape x_train120 per direzione: (1310, 120, 120, 1)
Shape x_test120 per direzione: (562, 120, 120, 1)
Shape y_train120 per direzione: (1310, 4)
Shape y_test120 per direzione: (562, 4)
Shape x_train120 per emozione: (1310, 120, 120, 1)
Shape x_test120 per emozione: (562, 120, 120, 1)
Shape y_train120 per

In [ ]:
# Richiamo della funzione 'train_and_test_reshape' per il reshape degli insiemi di train e test appena creati.
x_train30_direction, x_test30_direction, x_train60_direction, x_test60_direction, x_train120_direction, x_test120_direction = train_and_test_reshape(x_train30_direction, x_test30_direction, x_train60_direction, x_test60_direction, x_train120_direction, x_test120_direction, 'direzione')
x_train30_emotion, x_test30_emotion, x_train60_emotion, x_test60_emotion, x_train120_emotion, x_test120_emotion = train_and_test_reshape(x_train30_emotion, x_test30_emotion, x_train60_emotion, x_test60_emotion, x_train120_emotion, x_test120_emotion, 'emozione')

Prima del reshape per direzione
(1310, 30, 30, 1)
(562, 30, 30, 1)
(1310, 60, 60, 1)
(562, 60, 60, 1)
(1310, 120, 120, 1)
(562, 120, 120, 1)
Dopo il reshape per direzione
(1310, 900)
(562, 900)
(1310, 3600)
(562, 3600)
(1310, 14400)
(562, 14400)
Prima del reshape per emozione
(1310, 30, 30, 1)
(562, 30, 30, 1)
(1310, 60, 60, 1)
(562, 60, 60, 1)
(1310, 120, 120, 1)
(562, 120, 120, 1)
Dopo il reshape per emozione
(1310, 900)
(562, 900)
(1310, 3600)
(562, 3600)
(1310, 14400)
(562, 14400)


#Modelli per la classificazione della direzione del volto

##Modello 1

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- solver='adam';
- learning_rate='constant'.
"""
model30_direction_1 = MLPClassifier(hidden_layer_sizes=16, batch_size=50, max_iter=400, tol=0.00001, verbose=True)
model60_direction_1 = MLPClassifier(hidden_layer_sizes=16, batch_size=50, max_iter=400, tol=0.00001, verbose=True)
model120_direction_1 = MLPClassifier(hidden_layer_sizes=16, batch_size=50, max_iter=400, tol=0.00001, verbose=True)

history30_direction_1 = model_fitting(model30_direction_1, x_train30_direction, y_train30_direction, 0)
history60_direction_1 = model_fitting(model60_direction_1, x_train60_direction, y_train60_direction, 1)
history120_direction_1 = model_fitting(model120_direction_1, x_train120_direction, y_train120_direction, 2)

p_train30_direction_1, p_test30_direction_1 = predictions_realizing(model30_direction_1, x_train30_direction, x_test30_direction)
p_train60_direction_1, p_test60_direction_1 = predictions_realizing(model60_direction_1, x_train60_direction, x_test60_direction)
p_train120_direction_1, p_test120_direction_1 = predictions_realizing(model120_direction_1, x_train120_direction, x_test120_direction)

accuracy_train30_direction_1, accuracy_test30_direction_1 = score_history(y_train30_direction, p_train30_direction_1, y_test30_direction, p_test30_direction_1, 0)
accuracy_train60_direction_1, accuracy_test60_direction_1 = score_history(y_train60_direction, p_train60_direction_1, y_test60_direction, p_test60_direction_1, 1)
accuracy_train120_direction_1, accuracy_test120_direction_1 = score_history(y_train120_direction, p_train120_direction_1, y_test120_direction, p_test120_direction_1, 2)

Immagini piccole
Iteration 1, loss = 2.59780695
Iteration 2, loss = 2.16365238
Iteration 3, loss = 1.96688418
Iteration 4, loss = 1.79324890
Iteration 5, loss = 1.61792238
Iteration 6, loss = 1.45488033
Iteration 7, loss = 1.31485903
Iteration 8, loss = 1.19899402
Iteration 9, loss = 1.09120202
Iteration 10, loss = 0.99884903
Iteration 11, loss = 0.91906115
Iteration 12, loss = 0.83985544
Iteration 13, loss = 0.77984374
Iteration 14, loss = 0.71471492
Iteration 15, loss = 0.66222712
Iteration 16, loss = 0.61099675
Iteration 17, loss = 0.56373196
Iteration 18, loss = 0.52363811
Iteration 19, loss = 0.48337918
Iteration 20, loss = 0.45060031
Iteration 21, loss = 0.41985723
Iteration 22, loss = 0.38940218
Iteration 23, loss = 0.36456769
Iteration 24, loss = 0.33601239
Iteration 25, loss = 0.31027229
Iteration 26, loss = 0.28885455
Iteration 27, loss = 0.26925540
Iteration 28, loss = 0.25133619
Iteration 29, loss = 0.23561019
Iteration 30, loss = 0.21699042
Iteration 31, loss = 0.20386139


##Modello 2

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- learning_rate='constant';
- tol=0.0001.
"""
model30_direction_2 = MLPClassifier(hidden_layer_sizes=16, activation='identity', solver='sgd', batch_size=50, max_iter=500, verbose=True)
model60_direction_2 = MLPClassifier(hidden_layer_sizes=16, activation='identity', solver='sgd', batch_size=50, max_iter=500, verbose=True)
model120_direction_2 = MLPClassifier(hidden_layer_sizes=16, activation='identity', solver='sgd', batch_size=50, max_iter=500, verbose=True)

history30_direction_2 = model_fitting(model30_direction_2, x_train30_direction, y_train30_direction, 0)
history60_direction_2 = model_fitting(model60_direction_2, x_train60_direction, y_train60_direction, 1)
history120_direction_2 = model_fitting(model120_direction_2, x_train120_direction, y_train120_direction, 2)

p_train30_direction_2, p_test30_direction_2 = predictions_realizing(model30_direction_2, x_train30_direction, x_test30_direction)
p_train60_direction_2, p_test60_direction_2 = predictions_realizing(model60_direction_2, x_train60_direction, x_test60_direction)
p_train120_direction_2, p_test120_direction_2 = predictions_realizing(model120_direction_2, x_train120_direction, x_test120_direction)

accuracy_train30_direction_2, accuracy_test30_direction_2 = score_history(y_train30_direction, p_train30_direction_2, y_test30_direction, p_test30_direction_2, 0)
accuracy_train60_direction_2, accuracy_test60_direction_2 = score_history(y_train60_direction, p_train60_direction_2, y_test60_direction, p_test60_direction_2, 1)
accuracy_train120_direction_2, accuracy_test120_direction_2 = score_history(y_train120_direction, p_train120_direction_2, y_test120_direction, p_test120_direction_2, 2)

Immagini piccole
Iteration 1, loss = 2.65237021
Iteration 2, loss = 2.31435835
Iteration 3, loss = 2.18590634
Iteration 4, loss = 2.09060456
Iteration 5, loss = 2.00629541
Iteration 6, loss = 1.93630132
Iteration 7, loss = 1.87210884
Iteration 8, loss = 1.81530175
Iteration 9, loss = 1.76485565
Iteration 10, loss = 1.70868324
Iteration 11, loss = 1.65928388
Iteration 12, loss = 1.61571904
Iteration 13, loss = 1.57236845
Iteration 14, loss = 1.52950785
Iteration 15, loss = 1.49183396
Iteration 16, loss = 1.45298770
Iteration 17, loss = 1.41407963
Iteration 18, loss = 1.37908006
Iteration 19, loss = 1.34754063
Iteration 20, loss = 1.31329030
Iteration 21, loss = 1.28301361
Iteration 22, loss = 1.25118674
Iteration 23, loss = 1.22098184
Iteration 24, loss = 1.19417641
Iteration 25, loss = 1.16432045
Iteration 26, loss = 1.13856593
Iteration 27, loss = 1.11235496
Iteration 28, loss = 1.08703029
Iteration 29, loss = 1.06204056
Iteration 30, loss = 1.04052319
Iteration 31, loss = 1.02004730


##Modello 3

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- solver='adam';
- max_iter=200.
"""
model30_direction_3 = MLPClassifier(hidden_layer_sizes=32, batch_size=150, learning_rate='adaptive', tol=0.00001, verbose=True)
model60_direction_3 = MLPClassifier(hidden_layer_sizes=32, batch_size=150, learning_rate='adaptive', tol=0.00001, verbose=True)
model120_direction_3 = MLPClassifier(hidden_layer_sizes=32, batch_size=150, learning_rate='adaptive', tol=0.00001, verbose=True)

history30_direction_3 = model_fitting(model30_direction_3, x_train30_direction, y_train30_direction, 0)
history60_direction_3 = model_fitting(model60_direction_3, x_train60_direction, y_train60_direction, 1)
history120_direction_3 = model_fitting(model120_direction_3, x_train120_direction, y_train120_direction, 2)

p_train30_direction_3, p_test30_direction_3 = predictions_realizing(model30_direction_3, x_train30_direction, x_test30_direction)
p_train60_direction_3, p_test60_direction_3 = predictions_realizing(model60_direction_3, x_train60_direction, x_test60_direction)
p_train120_direction_3, p_test120_direction_3 = predictions_realizing(model120_direction_3, x_train120_direction, x_test120_direction)

accuracy_train30_direction_3, accuracy_test30_direction_3 = score_history(y_train30_direction, p_train30_direction_3, y_test30_direction, p_test30_direction_3, 0)
accuracy_train60_direction_3, accuracy_test60_direction_3 = score_history(y_train60_direction, p_train60_direction_3, y_test60_direction, p_test60_direction_3, 1)
accuracy_train120_direction_3, accuracy_test120_direction_3 = score_history(y_train120_direction, p_train120_direction_3, y_test120_direction, p_test120_direction_3, 2)

Immagini piccole
Iteration 1, loss = 2.42591343
Iteration 2, loss = 2.18462892
Iteration 3, loss = 2.04941252
Iteration 4, loss = 1.91159523
Iteration 5, loss = 1.77913279
Iteration 6, loss = 1.64401386
Iteration 7, loss = 1.51901832
Iteration 8, loss = 1.39725671
Iteration 9, loss = 1.29428098
Iteration 10, loss = 1.19258658
Iteration 11, loss = 1.11060387
Iteration 12, loss = 1.03262131
Iteration 13, loss = 0.96824778
Iteration 14, loss = 0.90487223
Iteration 15, loss = 0.84438462
Iteration 16, loss = 0.79349594
Iteration 17, loss = 0.74098503
Iteration 18, loss = 0.69740595
Iteration 19, loss = 0.65356240
Iteration 20, loss = 0.61633750
Iteration 21, loss = 0.58140415
Iteration 22, loss = 0.54530060
Iteration 23, loss = 0.51207278
Iteration 24, loss = 0.48445812
Iteration 25, loss = 0.45660545
Iteration 26, loss = 0.42859192
Iteration 27, loss = 0.40412842
Iteration 28, loss = 0.38211750
Iteration 29, loss = 0.35805714
Iteration 30, loss = 0.34250291
Iteration 31, loss = 0.32054666


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.34874551
Iteration 2, loss = 1.83147618
Iteration 3, loss = 1.44374810
Iteration 4, loss = 1.14315889
Iteration 5, loss = 0.92847391
Iteration 6, loss = 0.76081419
Iteration 7, loss = 0.63030046
Iteration 8, loss = 0.53115688
Iteration 9, loss = 0.45129527
Iteration 10, loss = 0.37228289
Iteration 11, loss = 0.32503223
Iteration 12, loss = 0.29084784
Iteration 13, loss = 0.24337178
Iteration 14, loss = 0.21037493
Iteration 15, loss = 0.17876620
Iteration 16, loss = 0.15446058
Iteration 17, loss = 0.13652042
Iteration 18, loss = 0.11521035
Iteration 19, loss = 0.10183699
Iteration 20, loss = 0.09119708
Iteration 21, loss = 0.08161388
Iteration 22, loss = 0.07403861
Iteration 23, loss = 0.06683009
Iteration 24, loss = 0.06088475
Iteration 25, loss = 0.05603934
Iteration 26, loss = 0.05167385
Iteration 27, loss = 0.04695941
Iteration 28, loss = 0.04331999
Iteration 29, loss = 0.04012782
Iteration 30, loss = 0.03723420
Iteration 31, loss = 0.03464050
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.6672597864768683
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.6174377224199288
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.6654804270462633


##Modello 4

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- learning_rate='constant'.
"""
model30_direction_4 = MLPClassifier(hidden_layer_sizes=32, activation='tanh', batch_size=150, max_iter=300, tol=0.00001, verbose=True)
model60_direction_4 = MLPClassifier(hidden_layer_sizes=32, activation='tanh', batch_size=150, max_iter=300, tol=0.00001, verbose=True)
model120_direction_4 = MLPClassifier(hidden_layer_sizes=32, activation='tanh', batch_size=150, max_iter=300, tol=0.00001, verbose=True)

history30_direction_4 = model_fitting(model30_direction_4, x_train30_direction, y_train30_direction, 0)
history60_direction_4 = model_fitting(model60_direction_4, x_train60_direction, y_train60_direction, 1)
history120_direction_4 = model_fitting(model120_direction_4, x_train120_direction, y_train120_direction, 2)

p_train30_direction_4, p_test30_direction_4 = predictions_realizing(model30_direction_4, x_train30_direction, x_test30_direction)
p_train60_direction_4, p_test60_direction_4 = predictions_realizing(model60_direction_4, x_train60_direction, x_test60_direction)
p_train120_direction_4, p_test120_direction_4 = predictions_realizing(model120_direction_4, x_train120_direction, x_test120_direction)

accuracy_train30_direction_4, accuracy_test30_direction_4 = score_history(y_train30_direction, p_train30_direction_4, y_test30_direction, p_test30_direction_4, 0)
accuracy_train60_direction_4, accuracy_test60_direction_4 = score_history(y_train60_direction, p_train60_direction_4, y_test60_direction, p_test60_direction_4, 1)
accuracy_train120_direction_4, accuracy_test120_direction_4 = score_history(y_train120_direction, p_train120_direction_4, y_test120_direction, p_test120_direction_4, 2)

Immagini piccole
Iteration 1, loss = 2.53730252
Iteration 2, loss = 2.26442304
Iteration 3, loss = 2.04153969
Iteration 4, loss = 1.91107583
Iteration 5, loss = 1.77991924
Iteration 6, loss = 1.67150368
Iteration 7, loss = 1.57157443
Iteration 8, loss = 1.48066733
Iteration 9, loss = 1.39752563
Iteration 10, loss = 1.32085763
Iteration 11, loss = 1.25172306
Iteration 12, loss = 1.18693538
Iteration 13, loss = 1.12594931
Iteration 14, loss = 1.06764854
Iteration 15, loss = 1.01502768
Iteration 16, loss = 0.96414486
Iteration 17, loss = 0.92103733
Iteration 18, loss = 0.87624500
Iteration 19, loss = 0.83610832
Iteration 20, loss = 0.79763917
Iteration 21, loss = 0.75789728
Iteration 22, loss = 0.72238261
Iteration 23, loss = 0.68498295
Iteration 24, loss = 0.65836190
Iteration 25, loss = 0.62815199
Iteration 26, loss = 0.60056880
Iteration 27, loss = 0.57135819
Iteration 28, loss = 0.54431485
Iteration 29, loss = 0.51957284
Iteration 30, loss = 0.49484946
Iteration 31, loss = 0.47357842


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 2, loss = 1.77638719
Iteration 3, loss = 1.45235889
Iteration 4, loss = 1.24188546
Iteration 5, loss = 1.06952089
Iteration 6, loss = 0.92769114
Iteration 7, loss = 0.81458279
Iteration 8, loss = 0.71807616
Iteration 9, loss = 0.63386439
Iteration 10, loss = 0.55987751
Iteration 11, loss = 0.49319336
Iteration 12, loss = 0.44845710
Iteration 13, loss = 0.39682715
Iteration 14, loss = 0.35574469
Iteration 15, loss = 0.31892904
Iteration 16, loss = 0.29255683
Iteration 17, loss = 0.26530424
Iteration 18, loss = 0.24247189
Iteration 19, loss = 0.22173561
Iteration 20, loss = 0.20524700
Iteration 21, loss = 0.19184464
Iteration 22, loss = 0.17712438
Iteration 23, loss = 0.16421878
Iteration 24, loss = 0.15340648
Iteration 25, loss = 0.14386629
Iteration 26, loss = 0.13488506
Iteration 27, loss = 0.12718500
Iteration 28, loss = 0.11980519
Iteration 29, loss = 0.11301105
Iteration 30, loss = 0.10676398
Iteration 31, loss = 0.10125344
Iteration 32, loss = 0.09612874
Iteration 33, lo

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 3.14913225
Iteration 2, loss = 2.81808626
Iteration 3, loss = 2.56276125
Iteration 4, loss = 2.34936206
Iteration 5, loss = 2.09614003
Iteration 6, loss = 1.95566128
Iteration 7, loss = 1.83475346
Iteration 8, loss = 1.73009174
Iteration 9, loss = 1.64009821
Iteration 10, loss = 1.55838349
Iteration 11, loss = 1.49368813
Iteration 12, loss = 1.43182257
Iteration 13, loss = 1.37812123
Iteration 14, loss = 1.33156346
Iteration 15, loss = 1.29335303
Iteration 16, loss = 1.25002705
Iteration 17, loss = 1.21903244
Iteration 18, loss = 1.18741174
Iteration 19, loss = 1.15520126
Iteration 20, loss = 1.12921759
Iteration 21, loss = 1.10136938
Iteration 22, loss = 1.07556407
Iteration 23, loss = 1.04789247
Iteration 24, loss = 1.02268882
Iteration 25, loss = 0.99607313
Iteration 26, loss = 0.97068680
Iteration 27, loss = 0.94763477
Iteration 28, loss = 0.92315351
Iteration 29, loss = 0.90076616
Iteration 30, loss = 0.87707945
Iteration 31, loss = 0.85338783
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.6886120996441281
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.6494661921708185
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.7348754448398577


##Modello 5

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- learning_rate='constant'.
"""
model30_direction_5 = MLPClassifier(hidden_layer_sizes=64, solver='sgd', batch_size=250, max_iter=900, tol=0.00001, verbose=True)
model60_direction_5 = MLPClassifier(hidden_layer_sizes=64, solver='sgd', batch_size=250, max_iter=900, tol=0.00001, verbose=True)
model120_direction_5 = MLPClassifier(hidden_layer_sizes=64, solver='sgd', batch_size=250, max_iter=900, tol=0.00001, verbose=True)

history30_direction_5 = model_fitting(model30_direction_5, x_train30_direction, y_train30_direction, 0)
history60_direction_5 = model_fitting(model60_direction_5, x_train60_direction, y_train60_direction, 1)
history120_direction_5 = model_fitting(model120_direction_5, x_train120_direction, y_train120_direction, 2)

p_train30_direction_5, p_test30_direction_5 = predictions_realizing(model30_direction_5, x_train30_direction, x_test30_direction)
p_train60_direction_5, p_test60_direction_5 = predictions_realizing(model60_direction_5, x_train60_direction, x_test60_direction)
p_train120_direction_5, p_test120_direction_5 = predictions_realizing(model120_direction_5, x_train120_direction, x_test120_direction)

accuracy_train30_direction_5, accuracy_test30_direction_5 = score_history(y_train30_direction, p_train30_direction_5, y_test30_direction, p_test30_direction_5, 0)
accuracy_train60_direction_5, accuracy_test60_direction_5 = score_history(y_train60_direction, p_train60_direction_5, y_test60_direction, p_test60_direction_5, 1)
accuracy_train120_direction_5, accuracy_test120_direction_5 = score_history(y_train120_direction, p_train120_direction_5, y_test120_direction, p_test120_direction_5, 2)

Immagini piccole
Iteration 1, loss = 3.00980928
Iteration 2, loss = 2.78362428
Iteration 3, loss = 2.55878665
Iteration 4, loss = 2.41158871
Iteration 5, loss = 2.33133343
Iteration 6, loss = 2.29312569
Iteration 7, loss = 2.27242942
Iteration 8, loss = 2.25602432
Iteration 9, loss = 2.24074894
Iteration 10, loss = 2.22607049
Iteration 11, loss = 2.21205650
Iteration 12, loss = 2.19887761
Iteration 13, loss = 2.18664299
Iteration 14, loss = 2.17481914
Iteration 15, loss = 2.16349597
Iteration 16, loss = 2.15231814
Iteration 17, loss = 2.14139701
Iteration 18, loss = 2.13061286
Iteration 19, loss = 2.11992555
Iteration 20, loss = 2.10956201
Iteration 21, loss = 2.09914096
Iteration 22, loss = 2.08908759
Iteration 23, loss = 2.07859085
Iteration 24, loss = 2.06835909
Iteration 25, loss = 2.05792224
Iteration 26, loss = 2.04803937
Iteration 27, loss = 2.03790238
Iteration 28, loss = 2.02790537
Iteration 29, loss = 2.01821077
Iteration 30, loss = 2.00833783
Iteration 31, loss = 1.99846948


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (900) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 2, loss = 2.45999901
Iteration 3, loss = 2.31600240
Iteration 4, loss = 2.27689531
Iteration 5, loss = 2.21894112
Iteration 6, loss = 2.16759613
Iteration 7, loss = 2.13069220
Iteration 8, loss = 2.09592985
Iteration 9, loss = 2.06218280
Iteration 10, loss = 2.02866221
Iteration 11, loss = 1.99586110
Iteration 12, loss = 1.96399457
Iteration 13, loss = 1.93237264
Iteration 14, loss = 1.90110882
Iteration 15, loss = 1.87094108
Iteration 16, loss = 1.84100946
Iteration 17, loss = 1.81102570
Iteration 18, loss = 1.78184761
Iteration 19, loss = 1.75290657
Iteration 20, loss = 1.72523666
Iteration 21, loss = 1.69786108
Iteration 22, loss = 1.67072527
Iteration 23, loss = 1.64363597
Iteration 24, loss = 1.61766618
Iteration 25, loss = 1.59155716
Iteration 26, loss = 1.56644876
Iteration 27, loss = 1.54129956
Iteration 28, loss = 1.51735304
Iteration 29, loss = 1.49350194
Iteration 30, loss = 1.46953385
Iteration 31, loss = 1.44710740
Iteration 32, loss = 1.42391989
Iteration 33, lo

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (900) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.64569192
Iteration 2, loss = 2.21406107
Iteration 3, loss = 1.98740895
Iteration 4, loss = 1.83027098
Iteration 5, loss = 1.69437837
Iteration 6, loss = 1.58094192
Iteration 7, loss = 1.47815782
Iteration 8, loss = 1.38485021
Iteration 9, loss = 1.30030683
Iteration 10, loss = 1.22727234
Iteration 11, loss = 1.15714074
Iteration 12, loss = 1.09468258
Iteration 13, loss = 1.04253132
Iteration 14, loss = 0.98862591
Iteration 15, loss = 0.93896146
Iteration 16, loss = 0.89293017
Iteration 17, loss = 0.84829266
Iteration 18, loss = 0.81392965
Iteration 19, loss = 0.77335234
Iteration 20, loss = 0.73858380
Iteration 21, loss = 0.70786080
Iteration 22, loss = 0.67662676
Iteration 23, loss = 0.64649080
Iteration 24, loss = 0.62144645
Iteration 25, loss = 0.59555152
Iteration 26, loss = 0.57216897
Iteration 27, loss = 0.54930183
Iteration 28, loss = 0.52697338
Iteration 29, loss = 0.50632700
Iteration 30, loss = 0.48819923
Iteration 31, loss = 0.47005957
Iteration 32, los

##Modello 6

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- tol=0.0001.
"""
model30_direction_6 = MLPClassifier(hidden_layer_sizes=64, activation='logistic', batch_size=250, learning_rate='adaptive', max_iter=700, verbose=True)
model60_direction_6 = MLPClassifier(hidden_layer_sizes=64, activation='logistic', batch_size=250, learning_rate='adaptive', max_iter=700, verbose=True)
model120_direction_6 = MLPClassifier(hidden_layer_sizes=64, activation='logistic', batch_size=250, learning_rate='adaptive', max_iter=700, verbose=True)

history30_direction_6 = model_fitting(model30_direction_6, x_train30_direction, y_train30_direction, 0)
history60_direction_6 = model_fitting(model60_direction_6, x_train60_direction, y_train60_direction, 1)
history120_direction_6 = model_fitting(model120_direction_6, x_train120_direction, y_train120_direction, 2)

p_train30_direction_6, p_test30_direction_6 = predictions_realizing(model30_direction_6, x_train30_direction, x_test30_direction)
p_train60_direction_6, p_test60_direction_6 = predictions_realizing(model60_direction_6, x_train60_direction, x_test60_direction)
p_train120_direction_6, p_test120_direction_6 = predictions_realizing(model120_direction_6, x_train120_direction, x_test120_direction)

accuracy_train30_direction_6, accuracy_test30_direction_6 = score_history(y_train30_direction, p_train30_direction_6, y_test30_direction, p_test30_direction_6, 0)
accuracy_train60_direction_6, accuracy_test60_direction_6 = score_history(y_train60_direction, p_train60_direction_6, y_test60_direction, p_test60_direction_6, 1)
accuracy_train120_direction_6, accuracy_test120_direction_6 = score_history(y_train120_direction, p_train120_direction_6, y_test120_direction, p_test120_direction_6, 2)

Immagini piccole
Iteration 1, loss = 2.38348554
Iteration 2, loss = 2.23095888
Iteration 3, loss = 2.21440371
Iteration 4, loss = 2.18232719
Iteration 5, loss = 2.14066272
Iteration 6, loss = 2.10488298
Iteration 7, loss = 2.07365164
Iteration 8, loss = 2.03909758
Iteration 9, loss = 2.00206263
Iteration 10, loss = 1.96815365
Iteration 11, loss = 1.93287344
Iteration 12, loss = 1.89558120
Iteration 13, loss = 1.85923220
Iteration 14, loss = 1.82164042
Iteration 15, loss = 1.78582580
Iteration 16, loss = 1.74656817
Iteration 17, loss = 1.70956348
Iteration 18, loss = 1.67181948
Iteration 19, loss = 1.63349355
Iteration 20, loss = 1.59631939
Iteration 21, loss = 1.55997915
Iteration 22, loss = 1.52199683
Iteration 23, loss = 1.48460274
Iteration 24, loss = 1.44895680
Iteration 25, loss = 1.41312191
Iteration 26, loss = 1.37826889
Iteration 27, loss = 1.34448977
Iteration 28, loss = 1.31023977
Iteration 29, loss = 1.27688767
Iteration 30, loss = 1.24513844
Iteration 31, loss = 1.21503414


##Modello 7

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- tol=0.0001.
"""
model30_direction_7 = MLPClassifier(hidden_layer_sizes=128, solver='sgd', batch_size=400, learning_rate='adaptive', max_iter=1000, verbose=True)
model60_direction_7 = MLPClassifier(hidden_layer_sizes=128, solver='sgd', batch_size=400, learning_rate='adaptive', max_iter=1000, verbose=True)
model120_direction_7 = MLPClassifier(hidden_layer_sizes=128, solver='sgd', batch_size=400, learning_rate='adaptive', max_iter=1000, verbose=True)

history30_direction_7 = model_fitting(model30_direction_7, x_train30_direction, y_train30_direction, 0)
history60_direction_7 = model_fitting(model60_direction_7, x_train60_direction, y_train60_direction, 1)
history120_direction_7 = model_fitting(model120_direction_7, x_train120_direction, y_train120_direction, 2)

p_train30_direction_7, p_test30_direction_7 = predictions_realizing(model30_direction_7, x_train30_direction, x_test30_direction)
p_train60_direction_7, p_test60_direction_7 = predictions_realizing(model60_direction_7, x_train60_direction, x_test60_direction)
p_train120_direction_7, p_test120_direction_7 = predictions_realizing(model120_direction_7, x_train120_direction, x_test120_direction)

accuracy_train30_direction_7, accuracy_test30_direction_7 = score_history(y_train30_direction, p_train30_direction_7, y_test30_direction, p_test30_direction_7, 0)
accuracy_train60_direction_7, accuracy_test60_direction_7 = score_history(y_train60_direction, p_train60_direction_7, y_test60_direction, p_test60_direction_7, 1)
accuracy_train120_direction_7, accuracy_test120_direction_7 = score_history(y_train120_direction, p_train120_direction_7, y_test120_direction, p_test120_direction_7, 2)

Immagini piccole
Iteration 1, loss = 3.12536055
Iteration 2, loss = 2.92133814
Iteration 3, loss = 2.69090111
Iteration 4, loss = 2.51643709
Iteration 5, loss = 2.40737294
Iteration 6, loss = 2.34506054
Iteration 7, loss = 2.31419695
Iteration 8, loss = 2.29570594
Iteration 9, loss = 2.28226135
Iteration 10, loss = 2.27004871
Iteration 11, loss = 2.25774610
Iteration 12, loss = 2.24592793
Iteration 13, loss = 2.23404510
Iteration 14, loss = 2.22311750
Iteration 15, loss = 2.21266304
Iteration 16, loss = 2.20307220
Iteration 17, loss = 2.19405306
Iteration 18, loss = 2.18541197
Iteration 19, loss = 2.17708708
Iteration 20, loss = 2.16908503
Iteration 21, loss = 2.16122038
Iteration 22, loss = 2.15339462
Iteration 23, loss = 2.14597857
Iteration 24, loss = 2.13842937
Iteration 25, loss = 2.13095351
Iteration 26, loss = 2.12363136
Iteration 27, loss = 2.11639020
Iteration 28, loss = 2.10914164
Iteration 29, loss = 2.10212947
Iteration 30, loss = 2.09507265
Iteration 31, loss = 2.08821689


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.94837528
Iteration 2, loss = 2.62567759
Iteration 3, loss = 2.40219748
Iteration 4, loss = 2.31400473
Iteration 5, loss = 2.27765756
Iteration 6, loss = 2.25043724
Iteration 7, loss = 2.21769451
Iteration 8, loss = 2.18436723
Iteration 9, loss = 2.15454630
Iteration 10, loss = 2.12770917
Iteration 11, loss = 2.10389322
Iteration 12, loss = 2.08119398
Iteration 13, loss = 2.06010783
Iteration 14, loss = 2.03925757
Iteration 15, loss = 2.01832285
Iteration 16, loss = 1.99874405
Iteration 17, loss = 1.97866997
Iteration 18, loss = 1.95923145
Iteration 19, loss = 1.93909437
Iteration 20, loss = 1.91948650
Iteration 21, loss = 1.89998371
Iteration 22, loss = 1.88085581
Iteration 23, loss = 1.86192066
Iteration 24, loss = 1.84289988
Iteration 25, loss = 1.82452589
Iteration 26, loss = 1.80596808
Iteration 27, loss = 1.78755243
Iteration 28, loss = 1.76930449
Iteration 29, loss = 1.75129504
Iteration 30, loss = 1.73320187
Iteration 31, loss = 1.71534373
Iteration 32, los

##Modello 8

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- max_iter=200.
"""
model30_direction_8 = MLPClassifier(hidden_layer_sizes=128, activation='identity', batch_size=400, learning_rate='invscaling', tol=0.00001, verbose=True)
model60_direction_8 = MLPClassifier(hidden_layer_sizes=128, activation='identity', batch_size=400, learning_rate='invscaling', tol=0.00001, verbose=True)
model120_direction_8 = MLPClassifier(hidden_layer_sizes=128, activation='identity', batch_size=400, learning_rate='invscaling', tol=0.00001, verbose=True)

history30_direction_8 = model_fitting(model30_direction_8, x_train30_direction, y_train30_direction, 0)
history60_direction_8 = model_fitting(model60_direction_8, x_train60_direction, y_train60_direction, 1)
history120_direction_8 = model_fitting(model120_direction_8, x_train120_direction, y_train120_direction, 2)

p_train30_direction_8, p_test30_direction_8 = predictions_realizing(model30_direction_8, x_train30_direction, x_test30_direction)
p_train60_direction_8, p_test60_direction_8 = predictions_realizing(model60_direction_8, x_train60_direction, x_test60_direction)
p_train120_direction_8, p_test120_direction_8 = predictions_realizing(model120_direction_8, x_train120_direction, x_test120_direction)

accuracy_train30_direction_8, accuracy_test30_direction_8 = score_history(y_train30_direction, p_train30_direction_8, y_test30_direction, p_test30_direction_8, 0)
accuracy_train60_direction_8, accuracy_test60_direction_8 = score_history(y_train60_direction, p_train60_direction_8, y_test60_direction, p_test60_direction_8, 1)
accuracy_train120_direction_8, accuracy_test120_direction_8 = score_history(y_train120_direction, p_train120_direction_8, y_test120_direction, p_test120_direction_8, 2)

Immagini piccole
Iteration 1, loss = 2.74695928
Iteration 2, loss = 2.36554420
Iteration 3, loss = 2.03189563
Iteration 4, loss = 1.79453487
Iteration 5, loss = 1.66077404
Iteration 6, loss = 1.53529146
Iteration 7, loss = 1.41382974
Iteration 8, loss = 1.33995617
Iteration 9, loss = 1.24606512
Iteration 10, loss = 1.17865846
Iteration 11, loss = 1.10974111
Iteration 12, loss = 1.06314574
Iteration 13, loss = 1.00342828
Iteration 14, loss = 0.94963039
Iteration 15, loss = 0.91072500
Iteration 16, loss = 0.86438086
Iteration 17, loss = 0.82732899
Iteration 18, loss = 0.78801926
Iteration 19, loss = 0.75617283
Iteration 20, loss = 0.71832191
Iteration 21, loss = 0.68903338
Iteration 22, loss = 0.66142246
Iteration 23, loss = 0.63163352
Iteration 24, loss = 0.60590087
Iteration 25, loss = 0.57931272
Iteration 26, loss = 0.55632115
Iteration 27, loss = 0.53250650
Iteration 28, loss = 0.50854460
Iteration 29, loss = 0.49468290
Iteration 30, loss = 0.47614682
Iteration 31, loss = 0.45323751


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 3.97837510
Iteration 2, loss = 2.47333522
Iteration 3, loss = 1.82756245
Iteration 4, loss = 1.60759019
Iteration 5, loss = 1.35656720
Iteration 6, loss = 1.08642083
Iteration 7, loss = 0.90673503
Iteration 8, loss = 0.80682701
Iteration 9, loss = 0.66164999
Iteration 10, loss = 0.57708282
Iteration 11, loss = 0.51497084
Iteration 12, loss = 0.46344689
Iteration 13, loss = 0.40105129
Iteration 14, loss = 0.35511564
Iteration 15, loss = 0.31783972
Iteration 16, loss = 0.27554613
Iteration 17, loss = 0.25559051
Iteration 18, loss = 0.22825180
Iteration 19, loss = 0.20873711
Iteration 20, loss = 0.18636004
Iteration 21, loss = 0.17326431
Iteration 22, loss = 0.16002831
Iteration 23, loss = 0.14365911
Iteration 24, loss = 0.13305996
Iteration 25, loss = 0.12233862
Iteration 26, loss = 0.11347479
Iteration 27, loss = 0.10557937
Iteration 28, loss = 0.09810884
Iteration 29, loss = 0.09263006
Iteration 30, loss = 0.08616107
Iteration 31, loss = 0.08144592
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 15.25030761
Iteration 2, loss = 15.04965891
Iteration 3, loss = 8.17757798
Iteration 4, loss = 5.56074248
Iteration 5, loss = 3.64105920
Iteration 6, loss = 2.45291725
Iteration 7, loss = 2.51600949
Iteration 8, loss = 1.51767948
Iteration 9, loss = 0.97036003
Iteration 10, loss = 0.81324257
Iteration 11, loss = 0.49093778
Iteration 12, loss = 0.31469413
Iteration 13, loss = 0.23933895
Iteration 14, loss = 0.18661312
Iteration 15, loss = 0.12244853
Iteration 16, loss = 0.08633979
Iteration 17, loss = 0.07686791
Iteration 18, loss = 0.06142904
Iteration 19, loss = 0.05084186
Iteration 20, loss = 0.04692875
Iteration 21, loss = 0.04339370
Iteration 22, loss = 0.03816281
Iteration 23, loss = 0.03551345
Iteration 24, loss = 0.03337226
Iteration 25, loss = 0.03150703
Iteration 26, loss = 0.02930911
Iteration 27, loss = 0.02848554
Iteration 28, loss = 0.02740091
Iteration 29, loss = 0.02629725
Iteration 30, loss = 0.02546256
Iteration 31, loss = 0.02473859
Iteration 32, l

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.6014234875444839
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.6423487544483986
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.6672597864768683


##Modello 9

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- solver='adam';
- max_iter=200;
- tol=0.0001.
"""
model30_direction_9 = MLPClassifier(hidden_layer_sizes=256, batch_size=500, learning_rate='invscaling', verbose=True)
model60_direction_9 = MLPClassifier(hidden_layer_sizes=256, batch_size=500, learning_rate='invscaling', verbose=True)
model120_direction_9 = MLPClassifier(hidden_layer_sizes=256, batch_size=500, learning_rate='invscaling', verbose=True)

history30_direction_9 = model_fitting(model30_direction_9, x_train30_direction, y_train30_direction, 0)
history60_direction_9 = model_fitting(model60_direction_9, x_train60_direction, y_train60_direction, 1)
history120_direction_9 = model_fitting(model120_direction_9, x_train120_direction, y_train120_direction, 2)

p_train30_direction_9, p_test30_direction_9 = predictions_realizing(model30_direction_9, x_train30_direction, x_test30_direction)
p_train60_direction_9, p_test60_direction_9 = predictions_realizing(model60_direction_9, x_train60_direction, x_test60_direction)
p_train120_direction_9, p_test120_direction_9 = predictions_realizing(model120_direction_9, x_train120_direction, x_test120_direction)

accuracy_train30_direction_9, accuracy_test30_direction_9 = score_history(y_train30_direction, p_train30_direction_9, y_test30_direction, p_test30_direction_9, 0)
accuracy_train60_direction_9, accuracy_test60_direction_9 = score_history(y_train60_direction, p_train60_direction_9, y_test60_direction, p_test60_direction_9, 1)
accuracy_train120_direction_9, accuracy_test120_direction_9 = score_history(y_train120_direction, p_train120_direction_9, y_test120_direction, p_test120_direction_9, 2)

Immagini piccole
Iteration 1, loss = 2.50338047
Iteration 2, loss = 2.22142009
Iteration 3, loss = 1.96051017
Iteration 4, loss = 1.82858104
Iteration 5, loss = 1.65445927
Iteration 6, loss = 1.50595675
Iteration 7, loss = 1.38308911
Iteration 8, loss = 1.25687832
Iteration 9, loss = 1.15373676
Iteration 10, loss = 1.05322229
Iteration 11, loss = 0.96301368
Iteration 12, loss = 0.88337062
Iteration 13, loss = 0.80709440
Iteration 14, loss = 0.74289322
Iteration 15, loss = 0.68097376
Iteration 16, loss = 0.62701507
Iteration 17, loss = 0.57751792
Iteration 18, loss = 0.53061643
Iteration 19, loss = 0.48873961
Iteration 20, loss = 0.45039303
Iteration 21, loss = 0.41599642
Iteration 22, loss = 0.38223884
Iteration 23, loss = 0.35525196
Iteration 24, loss = 0.32744629
Iteration 25, loss = 0.30367492
Iteration 26, loss = 0.28056655
Iteration 27, loss = 0.25986386
Iteration 28, loss = 0.24129191
Iteration 29, loss = 0.22361988
Iteration 30, loss = 0.20819755
Iteration 31, loss = 0.19382742


##Modello 10

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- max_iter=200;
- tol=0.0001.
"""
model30_direction_10 = MLPClassifier(hidden_layer_sizes=256, activation='identity', batch_size=500, learning_rate='adaptive', verbose=True)
model60_direction_10 = MLPClassifier(hidden_layer_sizes=256, activation='identity', batch_size=500, learning_rate='adaptive', verbose=True)
model120_direction_10 = MLPClassifier(hidden_layer_sizes=256, activation='identity', batch_size=500, learning_rate='adaptive', verbose=True)

history30_direction_10 = model_fitting(model30_direction_10, x_train30_direction, y_train30_direction, 0)
history60_direction_10 = model_fitting(model60_direction_10, x_train60_direction, y_train60_direction, 1)
history120_direction_10 = model_fitting(model120_direction_10, x_train120_direction, y_train120_direction, 2)

p_train30_direction_10, p_test30_direction_10 = predictions_realizing(model30_direction_10, x_train30_direction, x_test30_direction)
p_train60_direction_10, p_test60_direction_10 = predictions_realizing(model60_direction_10, x_train60_direction, x_test60_direction)
p_train120_direction_10, p_test120_direction_10 = predictions_realizing(model120_direction_10, x_train120_direction, x_test120_direction)

accuracy_train30_direction_10, accuracy_test30_direction_10 = score_history(y_train30_direction, p_train30_direction_10, y_test30_direction, p_test30_direction_10, 0)
accuracy_train60_direction_10, accuracy_test60_direction_10 = score_history(y_train60_direction, p_train60_direction_10, y_test60_direction, p_test60_direction_10, 1)
accuracy_train120_direction_10, accuracy_test120_direction_10 = score_history(y_train120_direction, p_train120_direction_10, y_test120_direction, p_test120_direction_10, 2)

Immagini piccole
Iteration 1, loss = 2.73668598
Iteration 2, loss = 2.41008197
Iteration 3, loss = 1.97391993
Iteration 4, loss = 1.80546404
Iteration 5, loss = 1.59454441
Iteration 6, loss = 1.51233136
Iteration 7, loss = 1.34069107
Iteration 8, loss = 1.28256191
Iteration 9, loss = 1.18237945
Iteration 10, loss = 1.10628231
Iteration 11, loss = 1.04159738
Iteration 12, loss = 0.98444571
Iteration 13, loss = 0.93292761
Iteration 14, loss = 0.88013942
Iteration 15, loss = 0.83670008
Iteration 16, loss = 0.79322960
Iteration 17, loss = 0.75252456
Iteration 18, loss = 0.72066205
Iteration 19, loss = 0.68203682
Iteration 20, loss = 0.65188315
Iteration 21, loss = 0.62874360
Iteration 22, loss = 0.59200231
Iteration 23, loss = 0.56849391
Iteration 24, loss = 0.54330312
Iteration 25, loss = 0.53109732
Iteration 26, loss = 0.49197148
Iteration 27, loss = 0.47437289
Iteration 28, loss = 0.45710345
Iteration 29, loss = 0.43310494
Iteration 30, loss = 0.41428967
Iteration 31, loss = 0.39665130


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 4.56546889
Iteration 2, loss = 2.91974353
Iteration 3, loss = 2.07592295
Iteration 4, loss = 2.13913893
Iteration 5, loss = 1.31455165
Iteration 6, loss = 1.29686798
Iteration 7, loss = 0.96998531
Iteration 8, loss = 0.87030108
Iteration 9, loss = 0.68672194
Iteration 10, loss = 0.64418098
Iteration 11, loss = 0.52803428
Iteration 12, loss = 0.47685315
Iteration 13, loss = 0.40494052
Iteration 14, loss = 0.36481482
Iteration 15, loss = 0.31911299
Iteration 16, loss = 0.28989803
Iteration 17, loss = 0.25235739
Iteration 18, loss = 0.23364541
Iteration 19, loss = 0.20728847
Iteration 20, loss = 0.19152632
Iteration 21, loss = 0.17389695
Iteration 22, loss = 0.15902769
Iteration 23, loss = 0.14734595
Iteration 24, loss = 0.13561775
Iteration 25, loss = 0.12576924
Iteration 26, loss = 0.11714702
Iteration 27, loss = 0.10939479
Iteration 28, loss = 0.10257956
Iteration 29, loss = 0.09601472
Iteration 30, loss = 0.09040927
Iteration 31, loss = 0.08484116
Iteration 32, los

#Modelli per la classificazione dell'espressione del volto

##Modello 1

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- solver='adam';
- learning_rate='constant'.
"""
model30_emotion_1 = MLPClassifier(hidden_layer_sizes=16, batch_size=50, max_iter=400, tol=0.00001, verbose=True)
model60_emotion_1 = MLPClassifier(hidden_layer_sizes=16, batch_size=50, max_iter=400, tol=0.00001, verbose=True)
model120_emotion_1 = MLPClassifier(hidden_layer_sizes=16, batch_size=50, max_iter=400, tol=0.00001, verbose=True)

history30_emotion_1 = model_fitting(model30_emotion_1, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_1 = model_fitting(model60_emotion_1, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_1 = model_fitting(model120_emotion_1, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_1, p_test30_emotion_1 = predictions_realizing(model30_emotion_1, x_train30_emotion, x_test30_emotion)
p_train60_emotion_1, p_test60_emotion_1 = predictions_realizing(model60_emotion_1, x_train60_emotion, x_test60_emotion)
p_train120_emotion_1, p_test120_emotion_1 = predictions_realizing(model120_emotion_1, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_1, accuracy_test30_emotion_1 = score_history(y_train30_emotion, p_train30_emotion_1, y_test30_emotion, p_test30_emotion_1, 0)
accuracy_train60_emotion_1, accuracy_test60_emotion_1 = score_history(y_train60_emotion, p_train60_emotion_1, y_test60_emotion, p_test60_emotion_1, 1)
accuracy_train120_emotion_1, accuracy_test120_emotion_1 = score_history(y_train120_emotion, p_train120_emotion_1, y_test120_emotion, p_test120_emotion_1, 2)

Immagini piccole
Iteration 1, loss = 2.34373833
Iteration 2, loss = 2.20358843
Iteration 3, loss = 2.11549023
Iteration 4, loss = 2.03533421
Iteration 5, loss = 1.95078205
Iteration 6, loss = 1.85477755
Iteration 7, loss = 1.75414419
Iteration 8, loss = 1.64643558
Iteration 9, loss = 1.55238656
Iteration 10, loss = 1.45775302
Iteration 11, loss = 1.36509724
Iteration 12, loss = 1.27784249
Iteration 13, loss = 1.20361607
Iteration 14, loss = 1.13274043
Iteration 15, loss = 1.06113073
Iteration 16, loss = 0.99970360
Iteration 17, loss = 0.93641555
Iteration 18, loss = 0.88653793
Iteration 19, loss = 0.82526036
Iteration 20, loss = 0.78013884
Iteration 21, loss = 0.73273610
Iteration 22, loss = 0.69150310
Iteration 23, loss = 0.65185521
Iteration 24, loss = 0.61261800
Iteration 25, loss = 0.57534092
Iteration 26, loss = 0.54265881
Iteration 27, loss = 0.51273831
Iteration 28, loss = 0.48360918
Iteration 29, loss = 0.45638208
Iteration 30, loss = 0.43322441
Iteration 31, loss = 0.40645125


##Modello 2

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- learning_rate='constant';
- tol=0.0001.
"""
model30_emotion_2 = MLPClassifier(hidden_layer_sizes=16, activation='identity', solver='sgd', batch_size=50, max_iter=500, verbose=True)
model60_emotion_2 = MLPClassifier(hidden_layer_sizes=16, activation='identity', solver='sgd', batch_size=50, max_iter=500, verbose=True)
model120_emotion_2 = MLPClassifier(hidden_layer_sizes=16, activation='identity', solver='sgd', batch_size=50, max_iter=500, verbose=True)

history30_emotion_2 = model_fitting(model30_emotion_2, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_2 = model_fitting(model60_emotion_2, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_2 = model_fitting(model120_emotion_2, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_2, p_test30_emotion_2 = predictions_realizing(model30_emotion_2, x_train30_emotion, x_test30_emotion)
p_train60_emotion_2, p_test60_emotion_2 = predictions_realizing(model60_emotion_2, x_train60_emotion, x_test60_emotion)
p_train120_emotion_2, p_test120_emotion_2 = predictions_realizing(model120_emotion_2, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_2, accuracy_test30_emotion_2 = score_history(y_train30_emotion, p_train30_emotion_2, y_test30_emotion, p_test30_emotion_2, 0)
accuracy_train60_emotion_2, accuracy_test60_emotion_2 = score_history(y_train60_emotion, p_train60_emotion_2, y_test60_emotion, p_test60_emotion_2, 1)
accuracy_train120_emotion_2, accuracy_test120_emotion_2 = score_history(y_train120_emotion, p_train120_emotion_2, y_test120_emotion, p_test120_emotion_2, 2)

Immagini piccole
Iteration 1, loss = 2.58962191
Iteration 2, loss = 2.35225694
Iteration 3, loss = 2.31058401
Iteration 4, loss = 2.27829435
Iteration 5, loss = 2.25024431
Iteration 6, loss = 2.22329177
Iteration 7, loss = 2.19849850
Iteration 8, loss = 2.17531398
Iteration 9, loss = 2.15281439
Iteration 10, loss = 2.13185565
Iteration 11, loss = 2.11082105
Iteration 12, loss = 2.09057111
Iteration 13, loss = 2.07338320
Iteration 14, loss = 2.05389325
Iteration 15, loss = 2.03373436
Iteration 16, loss = 2.01454598
Iteration 17, loss = 1.99417157
Iteration 18, loss = 1.97673856
Iteration 19, loss = 1.95900998
Iteration 20, loss = 1.94082570
Iteration 21, loss = 1.91993291
Iteration 22, loss = 1.90019386
Iteration 23, loss = 1.88310871
Iteration 24, loss = 1.86416807
Iteration 25, loss = 1.84456436
Iteration 26, loss = 1.82603982
Iteration 27, loss = 1.80662522
Iteration 28, loss = 1.78723111
Iteration 29, loss = 1.76892183
Iteration 30, loss = 1.75128957
Iteration 31, loss = 1.72979914


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.52873320
Iteration 2, loss = 2.24509485
Iteration 3, loss = 2.11296580
Iteration 4, loss = 1.99496384
Iteration 5, loss = 1.89232441
Iteration 6, loss = 1.79716256
Iteration 7, loss = 1.70489396
Iteration 8, loss = 1.62339581
Iteration 9, loss = 1.53596304
Iteration 10, loss = 1.45916711
Iteration 11, loss = 1.38194495
Iteration 12, loss = 1.30741208
Iteration 13, loss = 1.23909340
Iteration 14, loss = 1.17875545
Iteration 15, loss = 1.10135246
Iteration 16, loss = 1.04080281
Iteration 17, loss = 0.97898538
Iteration 18, loss = 0.92426716
Iteration 19, loss = 0.86858547
Iteration 20, loss = 0.81770386
Iteration 21, loss = 0.76910314
Iteration 22, loss = 0.72127895
Iteration 23, loss = 0.68022438
Iteration 24, loss = 0.63919073
Iteration 25, loss = 0.60042751
Iteration 26, loss = 0.56246154
Iteration 27, loss = 0.53020787
Iteration 28, loss = 0.50203675
Iteration 29, loss = 0.47227911
Iteration 30, loss = 0.44474654
Iteration 31, loss = 0.41908207
Iteration 32, los

##Modello 3

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- solver='adam';
- max_iter=200.
"""
model30_emotion_3 = MLPClassifier(hidden_layer_sizes=32, batch_size=150, learning_rate='adaptive', tol=0.00001, verbose=True)
model60_emotion_3 = MLPClassifier(hidden_layer_sizes=32, batch_size=150, learning_rate='adaptive', tol=0.00001, verbose=True)
model120_emotion_3 = MLPClassifier(hidden_layer_sizes=32, batch_size=150, learning_rate='adaptive', tol=0.00001, verbose=True)

history30_emotion_3 = model_fitting(model30_emotion_3, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_3 = model_fitting(model60_emotion_3, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_3 = model_fitting(model120_emotion_3, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_3, p_test30_emotion_3 = predictions_realizing(model30_emotion_3, x_train30_emotion, x_test30_emotion)
p_train60_emotion_3, p_test60_emotion_3 = predictions_realizing(model60_emotion_3, x_train60_emotion, x_test60_emotion)
p_train120_emotion_3, p_test120_emotion_3 = predictions_realizing(model120_emotion_3, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_3, accuracy_test30_emotion_3 = score_history(y_train30_emotion, p_train30_emotion_3, y_test30_emotion, p_test30_emotion_3, 0)
accuracy_train60_emotion_3, accuracy_test60_emotion_3 = score_history(y_train60_emotion, p_train60_emotion_3, y_test60_emotion, p_test60_emotion_3, 1)
accuracy_train120_emotion_3, accuracy_test120_emotion_3 = score_history(y_train120_emotion, p_train120_emotion_3, y_test120_emotion, p_test120_emotion_3, 2)

Immagini piccole
Iteration 1, loss = 2.34985151
Iteration 2, loss = 2.20534127
Iteration 3, loss = 2.11852315
Iteration 4, loss = 2.03134966
Iteration 5, loss = 1.95167019
Iteration 6, loss = 1.86307911
Iteration 7, loss = 1.77892294
Iteration 8, loss = 1.69734224
Iteration 9, loss = 1.61537487
Iteration 10, loss = 1.53397782
Iteration 11, loss = 1.45616170
Iteration 12, loss = 1.38254233
Iteration 13, loss = 1.31431610
Iteration 14, loss = 1.23893932
Iteration 15, loss = 1.16547622
Iteration 16, loss = 1.10007331
Iteration 17, loss = 1.03933063
Iteration 18, loss = 0.97714139
Iteration 19, loss = 0.92590860
Iteration 20, loss = 0.87429093
Iteration 21, loss = 0.82174161
Iteration 22, loss = 0.77385766
Iteration 23, loss = 0.72691957
Iteration 24, loss = 0.68322646
Iteration 25, loss = 0.64231715
Iteration 26, loss = 0.60606382
Iteration 27, loss = 0.57000124
Iteration 28, loss = 0.53802570
Iteration 29, loss = 0.50646007
Iteration 30, loss = 0.47684555
Iteration 31, loss = 0.45006958


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.44644805
Iteration 2, loss = 2.15108302
Iteration 3, loss = 1.92429711
Iteration 4, loss = 1.72109756
Iteration 5, loss = 1.51591896
Iteration 6, loss = 1.32693112
Iteration 7, loss = 1.17040266
Iteration 8, loss = 1.00744043
Iteration 9, loss = 0.88128449
Iteration 10, loss = 0.77228594
Iteration 11, loss = 0.68150614
Iteration 12, loss = 0.59602125
Iteration 13, loss = 0.53559831
Iteration 14, loss = 0.46191853
Iteration 15, loss = 0.40710934
Iteration 16, loss = 0.35237890
Iteration 17, loss = 0.30953690
Iteration 18, loss = 0.27049181
Iteration 19, loss = 0.24143836
Iteration 20, loss = 0.21317503
Iteration 21, loss = 0.19017908
Iteration 22, loss = 0.17087461
Iteration 23, loss = 0.14979756
Iteration 24, loss = 0.13458450
Iteration 25, loss = 0.11946922
Iteration 26, loss = 0.10897910
Iteration 27, loss = 0.09937795
Iteration 28, loss = 0.09101872
Iteration 29, loss = 0.08385117
Iteration 30, loss = 0.07706926
Iteration 31, loss = 0.07019743
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 3.32154429
Iteration 2, loss = 2.38874117
Iteration 3, loss = 2.11995285
Iteration 4, loss = 1.95037027
Iteration 5, loss = 1.82796557
Iteration 6, loss = 1.69791218
Iteration 7, loss = 1.56511724
Iteration 8, loss = 1.43198643
Iteration 9, loss = 1.31017669
Iteration 10, loss = 1.19364773
Iteration 11, loss = 1.09040965
Iteration 12, loss = 0.96805510
Iteration 13, loss = 0.86964847
Iteration 14, loss = 0.77025815
Iteration 15, loss = 0.68180468
Iteration 16, loss = 0.59564254
Iteration 17, loss = 0.51313707
Iteration 18, loss = 0.45203625
Iteration 19, loss = 0.39510894
Iteration 20, loss = 0.33696936
Iteration 21, loss = 0.29088137
Iteration 22, loss = 0.25560026
Iteration 23, loss = 0.22347143
Iteration 24, loss = 0.19715500
Iteration 25, loss = 0.17700512
Iteration 26, loss = 0.16002810
Iteration 27, loss = 0.14422984
Iteration 28, loss = 0.13117188
Iteration 29, loss = 0.11929588
Iteration 30, loss = 0.10958419
Iteration 31, loss = 0.10131846
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.5462633451957295
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.5409252669039146
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.5142348754448398


##Modello 4

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- learning_rate='constant'.
"""
model30_emotion_4 = MLPClassifier(hidden_layer_sizes=32, activation='tanh', batch_size=150, max_iter=300, tol=0.00001, verbose=True)
model60_emotion_4 = MLPClassifier(hidden_layer_sizes=32, activation='tanh', batch_size=150, max_iter=300, tol=0.00001, verbose=True)
model120_emotion_4 = MLPClassifier(hidden_layer_sizes=32, activation='tanh', batch_size=150, max_iter=300, tol=0.00001, verbose=True)

history30_emotion_4 = model_fitting(model30_emotion_4, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_4 = model_fitting(model60_emotion_4, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_4 = model_fitting(model120_emotion_4, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_4, p_test30_emotion_4 = predictions_realizing(model30_emotion_4, x_train30_emotion, x_test30_emotion)
p_train60_emotion_4, p_test60_emotion_4 = predictions_realizing(model60_emotion_4, x_train60_emotion, x_test60_emotion)
p_train120_emotion_4, p_test120_emotion_4 = predictions_realizing(model120_emotion_4, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_4, accuracy_test30_emotion_4 = score_history(y_train30_emotion, p_train30_emotion_4, y_test30_emotion, p_test30_emotion_4, 0)
accuracy_train60_emotion_4, accuracy_test60_emotion_4 = score_history(y_train60_emotion, p_train60_emotion_4, y_test60_emotion, p_test60_emotion_4, 1)
accuracy_train120_emotion_4, accuracy_test120_emotion_4 = score_history(y_train120_emotion, p_train120_emotion_4, y_test120_emotion, p_test120_emotion_4, 2)

Immagini piccole
Iteration 1, loss = 2.51709107
Iteration 2, loss = 2.29834435
Iteration 3, loss = 2.16690229
Iteration 4, loss = 2.06288495
Iteration 5, loss = 1.98604090
Iteration 6, loss = 1.90379934
Iteration 7, loss = 1.83323124
Iteration 8, loss = 1.76944375
Iteration 9, loss = 1.70721044
Iteration 10, loss = 1.64246262
Iteration 11, loss = 1.58685159
Iteration 12, loss = 1.53343015
Iteration 13, loss = 1.47827280
Iteration 14, loss = 1.42830713
Iteration 15, loss = 1.38193490
Iteration 16, loss = 1.33392808
Iteration 17, loss = 1.28782218
Iteration 18, loss = 1.24758470
Iteration 19, loss = 1.20676564
Iteration 20, loss = 1.16268858
Iteration 21, loss = 1.12845683
Iteration 22, loss = 1.09167307
Iteration 23, loss = 1.05068152
Iteration 24, loss = 1.01562019
Iteration 25, loss = 0.98831651
Iteration 26, loss = 0.95033417
Iteration 27, loss = 0.91894699
Iteration 28, loss = 0.89469403
Iteration 29, loss = 0.86054428
Iteration 30, loss = 0.83599986
Iteration 31, loss = 0.80588513


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.48942891
Iteration 2, loss = 2.06349397
Iteration 3, loss = 1.76129303
Iteration 4, loss = 1.51891569
Iteration 5, loss = 1.32494579
Iteration 6, loss = 1.15736348
Iteration 7, loss = 1.00158105
Iteration 8, loss = 0.88232708
Iteration 9, loss = 0.78164998
Iteration 10, loss = 0.68842196
Iteration 11, loss = 0.61697193
Iteration 12, loss = 0.55050285
Iteration 13, loss = 0.49761321
Iteration 14, loss = 0.45410319
Iteration 15, loss = 0.40957497
Iteration 16, loss = 0.36733502
Iteration 17, loss = 0.33530761
Iteration 18, loss = 0.30712159
Iteration 19, loss = 0.28264449
Iteration 20, loss = 0.26264671
Iteration 21, loss = 0.24135945
Iteration 22, loss = 0.22496591
Iteration 23, loss = 0.20938111
Iteration 24, loss = 0.19682263
Iteration 25, loss = 0.18291815
Iteration 26, loss = 0.17128076
Iteration 27, loss = 0.16161509
Iteration 28, loss = 0.15250707
Iteration 29, loss = 0.14319491
Iteration 30, loss = 0.13549378
Iteration 31, loss = 0.12812682
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.99021233
Iteration 2, loss = 2.74084908
Iteration 3, loss = 2.58548182
Iteration 4, loss = 2.42390763
Iteration 5, loss = 2.20837810
Iteration 6, loss = 2.07639154
Iteration 7, loss = 1.98654535
Iteration 8, loss = 1.91124771
Iteration 9, loss = 1.85339393
Iteration 10, loss = 1.79322320
Iteration 11, loss = 1.74935442
Iteration 12, loss = 1.71622354
Iteration 13, loss = 1.69042553
Iteration 14, loss = 1.66791867
Iteration 15, loss = 1.64874193
Iteration 16, loss = 1.62923031
Iteration 17, loss = 1.61154692
Iteration 18, loss = 1.59514756
Iteration 19, loss = 1.57752465
Iteration 20, loss = 1.56134198
Iteration 21, loss = 1.54516685
Iteration 22, loss = 1.53047322
Iteration 23, loss = 1.51613254
Iteration 24, loss = 1.50197936
Iteration 25, loss = 1.48876204
Iteration 26, loss = 1.47532612
Iteration 27, loss = 1.46222461
Iteration 28, loss = 1.44965273
Iteration 29, loss = 1.43716153
Iteration 30, loss = 1.42476643
Iteration 31, loss = 1.41328789
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.5729537366548043
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.5533807829181495
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.5569395017793595


##Modello 5

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- learning_rate='constant'.
"""
model30_emotion_5 = MLPClassifier(hidden_layer_sizes=64, solver='sgd', batch_size=250, max_iter=900, tol=0.00001, verbose=True)
model60_emotion_5 = MLPClassifier(hidden_layer_sizes=64, solver='sgd', batch_size=250, max_iter=900, tol=0.00001, verbose=True)
model120_emotion_5 = MLPClassifier(hidden_layer_sizes=64, solver='sgd', batch_size=250, max_iter=900, tol=0.00001, verbose=True)

history30_emotion_5 = model_fitting(model30_emotion_5, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_5 = model_fitting(model60_emotion_5, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_5 = model_fitting(model120_emotion_5, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_5, p_test30_emotion_5 = predictions_realizing(model30_emotion_5, x_train30_emotion, x_test30_emotion)
p_train60_emotion_5, p_test60_emotion_5 = predictions_realizing(model60_emotion_5, x_train60_emotion, x_test60_emotion)
p_train120_emotion_5, p_test120_emotion_5 = predictions_realizing(model120_emotion_5, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_5, accuracy_test30_emotion_5 = score_history(y_train30_emotion, p_train30_emotion_5, y_test30_emotion, p_test30_emotion_5, 0)
accuracy_train60_emotion_5, accuracy_test60_emotion_5 = score_history(y_train60_emotion, p_train60_emotion_5, y_test60_emotion, p_test60_emotion_5, 1)
accuracy_train120_emotion_5, accuracy_test120_emotion_5 = score_history(y_train120_emotion, p_train120_emotion_5, y_test120_emotion, p_test120_emotion_5, 2)

Immagini piccole
Iteration 1, loss = 2.95127109
Iteration 2, loss = 2.74273663
Iteration 3, loss = 2.54637822
Iteration 4, loss = 2.41855929
Iteration 5, loss = 2.34426010
Iteration 6, loss = 2.30489771
Iteration 7, loss = 2.28498153
Iteration 8, loss = 2.27280262
Iteration 9, loss = 2.26388524
Iteration 10, loss = 2.25597973
Iteration 11, loss = 2.24859018
Iteration 12, loss = 2.24152280
Iteration 13, loss = 2.23497710
Iteration 14, loss = 2.22877573
Iteration 15, loss = 2.22300164
Iteration 16, loss = 2.21742619
Iteration 17, loss = 2.21225119
Iteration 18, loss = 2.20719278
Iteration 19, loss = 2.20214965
Iteration 20, loss = 2.19722341
Iteration 21, loss = 2.19255559
Iteration 22, loss = 2.18780944
Iteration 23, loss = 2.18334474
Iteration 24, loss = 2.17872288
Iteration 25, loss = 2.17426743
Iteration 26, loss = 2.16991247
Iteration 27, loss = 2.16536043
Iteration 28, loss = 2.16106158
Iteration 29, loss = 2.15681930
Iteration 30, loss = 2.15248767
Iteration 31, loss = 2.14826597


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (900) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.92306140
Iteration 2, loss = 2.51120624
Iteration 3, loss = 2.35074594
Iteration 4, loss = 2.29498535
Iteration 5, loss = 2.25833290
Iteration 6, loss = 2.23210153
Iteration 7, loss = 2.21159073
Iteration 8, loss = 2.19058305
Iteration 9, loss = 2.17172509
Iteration 10, loss = 2.15391945
Iteration 11, loss = 2.13742191
Iteration 12, loss = 2.12063858
Iteration 13, loss = 2.10464056
Iteration 14, loss = 2.08803037
Iteration 15, loss = 2.07138465
Iteration 16, loss = 2.05507070
Iteration 17, loss = 2.03904421
Iteration 18, loss = 2.02302459
Iteration 19, loss = 2.00752382
Iteration 20, loss = 1.99138388
Iteration 21, loss = 1.97644587
Iteration 22, loss = 1.96074920
Iteration 23, loss = 1.94507482
Iteration 24, loss = 1.92887867
Iteration 25, loss = 1.91327481
Iteration 26, loss = 1.89859381
Iteration 27, loss = 1.88262040
Iteration 28, loss = 1.86713518
Iteration 29, loss = 1.85116627
Iteration 30, loss = 1.83585143
Iteration 31, loss = 1.81962098
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (900) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.55514930
Iteration 2, loss = 2.31003344
Iteration 3, loss = 2.20762338
Iteration 4, loss = 2.14273196
Iteration 5, loss = 2.06992142
Iteration 6, loss = 1.99482470
Iteration 7, loss = 1.93326047
Iteration 8, loss = 1.85870267
Iteration 9, loss = 1.79340407
Iteration 10, loss = 1.71954086
Iteration 11, loss = 1.65315370
Iteration 12, loss = 1.58599995
Iteration 13, loss = 1.52399550
Iteration 14, loss = 1.46511245
Iteration 15, loss = 1.40688109
Iteration 16, loss = 1.35676409
Iteration 17, loss = 1.29749273
Iteration 18, loss = 1.24624363
Iteration 19, loss = 1.20054559
Iteration 20, loss = 1.15081753
Iteration 21, loss = 1.10418273
Iteration 22, loss = 1.05803750
Iteration 23, loss = 1.01651776
Iteration 24, loss = 0.97618664
Iteration 25, loss = 0.93654910
Iteration 26, loss = 0.90122103
Iteration 27, loss = 0.86578641
Iteration 28, loss = 0.83123377
Iteration 29, loss = 0.79894092
Iteration 30, loss = 0.76749354
Iteration 31, loss = 0.73707574
Iteration 32, los

##Modello 6

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- tol=0.0001.
"""
model30_emotion_6 = MLPClassifier(hidden_layer_sizes=64, activation='logistic', batch_size=250, learning_rate='adaptive', max_iter=700, verbose=True)
model60_emotion_6 = MLPClassifier(hidden_layer_sizes=64, activation='logistic', batch_size=250, learning_rate='adaptive', max_iter=700, verbose=True)
model120_emotion_6 = MLPClassifier(hidden_layer_sizes=64, activation='logistic', batch_size=250, learning_rate='adaptive', max_iter=700, verbose=True)

history30_emotion_6 = model_fitting(model30_emotion_6, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_6 = model_fitting(model60_emotion_6, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_6 = model_fitting(model120_emotion_6, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_6, p_test30_emotion_6 = predictions_realizing(model30_emotion_6, x_train30_emotion, x_test30_emotion)
p_train60_emotion_6, p_test60_emotion_6 = predictions_realizing(model60_emotion_6, x_train60_emotion, x_test60_emotion)
p_train120_emotion_6, p_test120_emotion_6 = predictions_realizing(model120_emotion_6, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_6, accuracy_test30_emotion_6 = score_history(y_train30_emotion, p_train30_emotion_6, y_test30_emotion, p_test30_emotion_6, 0)
accuracy_train60_emotion_6, accuracy_test60_emotion_6 = score_history(y_train60_emotion, p_train60_emotion_6, y_test60_emotion, p_test60_emotion_6, 1)
accuracy_train120_emotion_6, accuracy_test120_emotion_6 = score_history(y_train120_emotion, p_train120_emotion_6, y_test120_emotion, p_test120_emotion_6, 2)

Immagini piccole
Iteration 1, loss = 2.70490245
Iteration 2, loss = 2.33037351
Iteration 3, loss = 2.24577162
Iteration 4, loss = 2.22789122
Iteration 5, loss = 2.22075428
Iteration 6, loss = 2.20894814
Iteration 7, loss = 2.19278794
Iteration 8, loss = 2.17544395
Iteration 9, loss = 2.15941315
Iteration 10, loss = 2.14495430
Iteration 11, loss = 2.12836519
Iteration 12, loss = 2.11230768
Iteration 13, loss = 2.09468162
Iteration 14, loss = 2.07675855
Iteration 15, loss = 2.05895848
Iteration 16, loss = 2.04063562
Iteration 17, loss = 2.02196805
Iteration 18, loss = 2.00406389
Iteration 19, loss = 1.98282767
Iteration 20, loss = 1.96127639
Iteration 21, loss = 1.94240437
Iteration 22, loss = 1.92158360
Iteration 23, loss = 1.89974723
Iteration 24, loss = 1.87799557
Iteration 25, loss = 1.85704213
Iteration 26, loss = 1.83533676
Iteration 27, loss = 1.81316740
Iteration 28, loss = 1.79087147
Iteration 29, loss = 1.76859979
Iteration 30, loss = 1.74470498
Iteration 31, loss = 1.72189475


##Modello 7

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- tol=0.0001.
"""
model30_emotion_7 = MLPClassifier(hidden_layer_sizes=128, solver='sgd', batch_size=400, learning_rate='adaptive', max_iter=1000, verbose=True)
model60_emotion_7 = MLPClassifier(hidden_layer_sizes=128, solver='sgd', batch_size=400, learning_rate='adaptive', max_iter=1000, verbose=True)
model120_emotion_7 = MLPClassifier(hidden_layer_sizes=128, solver='sgd', batch_size=400, learning_rate='adaptive', max_iter=1000, verbose=True)

history30_emotion_7 = model_fitting(model30_emotion_7, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_7 = model_fitting(model60_emotion_7, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_7 = model_fitting(model120_emotion_7, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_7, p_test30_emotion_7 = predictions_realizing(model30_emotion_7, x_train30_emotion, x_test30_emotion)
p_train60_emotion_7, p_test60_emotion_7 = predictions_realizing(model60_emotion_7, x_train60_emotion, x_test60_emotion)
p_train120_emotion_7, p_test120_emotion_7 = predictions_realizing(model120_emotion_7, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_7, accuracy_test30_emotion_7 = score_history(y_train30_emotion, p_train30_emotion_7, y_test30_emotion, p_test30_emotion_7, 0)
accuracy_train60_emotion_7, accuracy_test60_emotion_7 = score_history(y_train60_emotion, p_train60_emotion_7, y_test60_emotion, p_test60_emotion_7, 1)
accuracy_train120_emotion_7, accuracy_test120_emotion_7 = score_history(y_train120_emotion, p_train120_emotion_7, y_test120_emotion, p_test120_emotion_7, 2)

Immagini piccole
Iteration 1, loss = 2.56885294
Iteration 2, loss = 2.51396041
Iteration 3, loss = 2.44688658
Iteration 4, loss = 2.38585278
Iteration 5, loss = 2.34117688
Iteration 6, loss = 2.31172662
Iteration 7, loss = 2.29460334
Iteration 8, loss = 2.28486704
Iteration 9, loss = 2.27885602
Iteration 10, loss = 2.27424312
Iteration 11, loss = 2.27011302
Iteration 12, loss = 2.26568319
Iteration 13, loss = 2.26134083
Iteration 14, loss = 2.25661493
Iteration 15, loss = 2.25214260
Iteration 16, loss = 2.24770395
Iteration 17, loss = 2.24362633
Iteration 18, loss = 2.23952223
Iteration 19, loss = 2.23575019
Iteration 20, loss = 2.23187929
Iteration 21, loss = 2.22814221
Iteration 22, loss = 2.22445169
Iteration 23, loss = 2.22087320
Iteration 24, loss = 2.21743504
Iteration 25, loss = 2.21392701
Iteration 26, loss = 2.21036389
Iteration 27, loss = 2.20679392
Iteration 28, loss = 2.20330470
Iteration 29, loss = 2.19980776
Iteration 30, loss = 2.19651393
Iteration 31, loss = 2.19306576


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 2.94534697
Iteration 2, loss = 2.63032201
Iteration 3, loss = 2.42777659
Iteration 4, loss = 2.35652872
Iteration 5, loss = 2.33597623
Iteration 6, loss = 2.31474746
Iteration 7, loss = 2.28313758
Iteration 8, loss = 2.25002900
Iteration 9, loss = 2.22315720
Iteration 10, loss = 2.20304975
Iteration 11, loss = 2.18661557
Iteration 12, loss = 2.17137784
Iteration 13, loss = 2.15584592
Iteration 14, loss = 2.14082992
Iteration 15, loss = 2.12620540
Iteration 16, loss = 2.11184108
Iteration 17, loss = 2.09784265
Iteration 18, loss = 2.08398539
Iteration 19, loss = 2.07060021
Iteration 20, loss = 2.05752495
Iteration 21, loss = 2.04434843
Iteration 22, loss = 2.03161472
Iteration 23, loss = 2.01820212
Iteration 24, loss = 2.00495565
Iteration 25, loss = 1.99209566
Iteration 26, loss = 1.97954615
Iteration 27, loss = 1.96712994
Iteration 28, loss = 1.95432338
Iteration 29, loss = 1.94184945
Iteration 30, loss = 1.92921127
Iteration 31, loss = 1.91708813
Iteration 32, los

##Modello 8

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- max_iter=200.
"""
model30_emotion_8 = MLPClassifier(hidden_layer_sizes=128, activation='identity', batch_size=400, learning_rate='invscaling', tol=0.00001, verbose=True)
model60_emotion_8 = MLPClassifier(hidden_layer_sizes=128, activation='identity', batch_size=400, learning_rate='invscaling', tol=0.00001, verbose=True)
model120_emotion_8 = MLPClassifier(hidden_layer_sizes=128, activation='identity', batch_size=400, learning_rate='invscaling', tol=0.00001, verbose=True)

history30_emotion_8 = model_fitting(model30_emotion_8, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_8 = model_fitting(model60_emotion_8, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_8 = model_fitting(model120_emotion_8, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_8, p_test30_emotion_8 = predictions_realizing(model30_emotion_8, x_train30_emotion, x_test30_emotion)
p_train60_emotion_8, p_test60_emotion_8 = predictions_realizing(model60_emotion_8, x_train60_emotion, x_test60_emotion)
p_train120_emotion_8, p_test120_emotion_8 = predictions_realizing(model120_emotion_8, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_8, accuracy_test30_emotion_8 = score_history(y_train30_emotion, p_train30_emotion_8, y_test30_emotion, p_test30_emotion_8, 0)
accuracy_train60_emotion_8, accuracy_test60_emotion_8 = score_history(y_train60_emotion, p_train60_emotion_8, y_test60_emotion, p_test60_emotion_8, 1)
accuracy_train120_emotion_8, accuracy_test120_emotion_8 = score_history(y_train120_emotion, p_train120_emotion_8, y_test120_emotion, p_test120_emotion_8, 2)

Immagini piccole
Iteration 1, loss = 2.67088254
Iteration 2, loss = 2.43028402
Iteration 3, loss = 2.21064138
Iteration 4, loss = 2.05573546
Iteration 5, loss = 1.98554492
Iteration 6, loss = 1.86970493
Iteration 7, loss = 1.81962766
Iteration 8, loss = 1.74001897
Iteration 9, loss = 1.67566032
Iteration 10, loss = 1.61095746
Iteration 11, loss = 1.55982986
Iteration 12, loss = 1.50364543
Iteration 13, loss = 1.46039892
Iteration 14, loss = 1.41329637
Iteration 15, loss = 1.37222518
Iteration 16, loss = 1.33924263
Iteration 17, loss = 1.30205855
Iteration 18, loss = 1.25990572
Iteration 19, loss = 1.22671808
Iteration 20, loss = 1.19623514
Iteration 21, loss = 1.16574841
Iteration 22, loss = 1.13518762
Iteration 23, loss = 1.10570560
Iteration 24, loss = 1.07632693
Iteration 25, loss = 1.04870943
Iteration 26, loss = 1.02033467
Iteration 27, loss = 1.00095534
Iteration 28, loss = 0.97892659
Iteration 29, loss = 0.95437496
Iteration 30, loss = 0.92942768
Iteration 31, loss = 0.90506391


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 3.76635398
Iteration 2, loss = 2.73433531
Iteration 3, loss = 2.04795888
Iteration 4, loss = 1.84320068
Iteration 5, loss = 1.58307416
Iteration 6, loss = 1.35746509
Iteration 7, loss = 1.13628709
Iteration 8, loss = 0.97353936
Iteration 9, loss = 0.88671135
Iteration 10, loss = 0.79592929
Iteration 11, loss = 0.69301553
Iteration 12, loss = 0.61695969
Iteration 13, loss = 0.55860593
Iteration 14, loss = 0.49496871
Iteration 15, loss = 0.44723083
Iteration 16, loss = 0.40092602
Iteration 17, loss = 0.36248078
Iteration 18, loss = 0.32389724
Iteration 19, loss = 0.29731689
Iteration 20, loss = 0.26996875
Iteration 21, loss = 0.24446440
Iteration 22, loss = 0.22662590
Iteration 23, loss = 0.20519189
Iteration 24, loss = 0.18978126
Iteration 25, loss = 0.17686464
Iteration 26, loss = 0.16310809
Iteration 27, loss = 0.15166639
Iteration 28, loss = 0.14120421
Iteration 29, loss = 0.13064245
Iteration 30, loss = 0.12273389
Iteration 31, loss = 0.11475581
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 13.46946692
Iteration 2, loss = 13.39136382
Iteration 3, loss = 9.42808646
Iteration 4, loss = 6.90091681
Iteration 5, loss = 8.25490458
Iteration 6, loss = 3.79477787
Iteration 7, loss = 2.74898357
Iteration 8, loss = 2.46926155
Iteration 9, loss = 1.60154284
Iteration 10, loss = 1.62528176
Iteration 11, loss = 0.94727132
Iteration 12, loss = 0.91659542
Iteration 13, loss = 0.51942311
Iteration 14, loss = 0.44590816
Iteration 15, loss = 0.29985857
Iteration 16, loss = 0.24199560
Iteration 17, loss = 0.21991772
Iteration 18, loss = 0.19299596
Iteration 19, loss = 0.16459817
Iteration 20, loss = 0.13272738
Iteration 21, loss = 0.11966950
Iteration 22, loss = 0.11416311
Iteration 23, loss = 0.10599448
Iteration 24, loss = 0.09781436
Iteration 25, loss = 0.09267703
Iteration 26, loss = 0.08653583
Iteration 27, loss = 0.08211384
Iteration 28, loss = 0.07867577
Iteration 29, loss = 0.07575274
Iteration 30, loss = 0.07252231
Iteration 31, loss = 0.06947742
Iteration 32, l

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.5391459074733096
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.5391459074733096
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.5106761565836299


##Modello 9

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- activation='relu';
- solver='adam';
- max_iter=200;
- tol=0.0001.
"""
model30_emotion_9 = MLPClassifier(hidden_layer_sizes=256, batch_size=500, learning_rate='invscaling', verbose=True)
model60_emotion_9 = MLPClassifier(hidden_layer_sizes=256, batch_size=500, learning_rate='invscaling', verbose=True)
model120_emotion_9 = MLPClassifier(hidden_layer_sizes=256, batch_size=500, learning_rate='invscaling', verbose=True)

history30_emotion_9 = model_fitting(model30_emotion_9, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_9 = model_fitting(model60_emotion_9, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_9 = model_fitting(model120_emotion_9, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_9, p_test30_emotion_9 = predictions_realizing(model30_emotion_9, x_train30_emotion, x_test30_emotion)
p_train60_emotion_9, p_test60_emotion_9 = predictions_realizing(model60_emotion_9, x_train60_emotion, x_test60_emotion)
p_train120_emotion_9, p_test120_emotion_9 = predictions_realizing(model120_emotion_9, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_9, accuracy_test30_emotion_9 = score_history(y_train30_emotion, p_train30_emotion_9, y_test30_emotion, p_test30_emotion_9, 0)
accuracy_train60_emotion_9, accuracy_test60_emotion_9 = score_history(y_train60_emotion, p_train60_emotion_9, y_test60_emotion, p_test60_emotion_9, 1)
accuracy_train120_emotion_9, accuracy_test120_emotion_9 = score_history(y_train120_emotion, p_train120_emotion_9, y_test120_emotion, p_test120_emotion_9, 2)

Immagini piccole
Iteration 1, loss = 2.54379933
Iteration 2, loss = 2.34296764
Iteration 3, loss = 2.13145764
Iteration 4, loss = 2.03833859
Iteration 5, loss = 1.96572121
Iteration 6, loss = 1.85266961
Iteration 7, loss = 1.76246232
Iteration 8, loss = 1.68179431
Iteration 9, loss = 1.58394551
Iteration 10, loss = 1.50828199
Iteration 11, loss = 1.42346636
Iteration 12, loss = 1.34624263
Iteration 13, loss = 1.27044983
Iteration 14, loss = 1.19549252
Iteration 15, loss = 1.12256052
Iteration 16, loss = 1.05450757
Iteration 17, loss = 0.99088826
Iteration 18, loss = 0.92641171
Iteration 19, loss = 0.87004895
Iteration 20, loss = 0.81243878
Iteration 21, loss = 0.75858997
Iteration 22, loss = 0.70859259
Iteration 23, loss = 0.66104716
Iteration 24, loss = 0.61613346
Iteration 25, loss = 0.57498544
Iteration 26, loss = 0.53523026
Iteration 27, loss = 0.49914546
Iteration 28, loss = 0.46611158
Iteration 29, loss = 0.43392843
Iteration 30, loss = 0.40583112
Iteration 31, loss = 0.37764053


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Train score modello per immagini piccole: 1.0
Test score modello per immagini piccole: 0.5142348754448398
Train score modello per immagini medie: 1.0
Test score modello per immagini medie: 0.5373665480427047
Train score modello per immagini grandi: 1.0
Test score modello per immagini grandi: 0.5088967971530249


##Modello 10

In [ ]:
"""
Richiamo delle funzioni 'model_fitting', 'predictions_realizing' e 'score_history' per la classificazione attraverso questo modello.
I parametri principali impostati con valori di default in 'MLPClassifier' sono:
- solver='adam';
- max_iter=200;
- tol=0.0001.
"""
model30_emotion_10 = MLPClassifier(hidden_layer_sizes=256, activation='identity', batch_size=500, learning_rate='adaptive', verbose=True)
model60_emotion_10 = MLPClassifier(hidden_layer_sizes=256, activation='identity', batch_size=500, learning_rate='adaptive', verbose=True)
model120_emotion_10 = MLPClassifier(hidden_layer_sizes=256, activation='identity', batch_size=500, learning_rate='adaptive', verbose=True)

history30_emotion_10 = model_fitting(model30_emotion_10, x_train30_emotion, y_train30_emotion, 0)
history60_emotion_10 = model_fitting(model60_emotion_10, x_train60_emotion, y_train60_emotion, 1)
history120_emotion_10 = model_fitting(model120_emotion_10, x_train120_emotion, y_train120_emotion, 2)

p_train30_emotion_10, p_test30_emotion_10 = predictions_realizing(model30_emotion_10, x_train30_emotion, x_test30_emotion)
p_train60_emotion_10, p_test60_emotion_10 = predictions_realizing(model60_emotion_10, x_train60_emotion, x_test60_emotion)
p_train120_emotion_10, p_test120_emotion_10 = predictions_realizing(model120_emotion_10, x_train120_emotion, x_test120_emotion)

accuracy_train30_emotion_10, accuracy_test30_emotion_10 = score_history(y_train30_emotion, p_train30_emotion_10, y_test30_emotion, p_test30_emotion_10, 0)
accuracy_train60_emotion_10, accuracy_test60_emotion_10 = score_history(y_train60_emotion, p_train60_emotion_10, y_test60_emotion, p_test60_emotion_10, 1)
accuracy_train120_emotion_10, accuracy_test120_emotion_10 = score_history(y_train120_emotion, p_train120_emotion_10, y_test120_emotion, p_test120_emotion_10, 2)

Immagini piccole
Iteration 1, loss = 2.62818799
Iteration 2, loss = 2.30246781
Iteration 3, loss = 2.13964643
Iteration 4, loss = 2.04152653
Iteration 5, loss = 1.88816258
Iteration 6, loss = 1.81224840
Iteration 7, loss = 1.70678304
Iteration 8, loss = 1.64230445
Iteration 9, loss = 1.57059946
Iteration 10, loss = 1.50550373
Iteration 11, loss = 1.45513999
Iteration 12, loss = 1.39860969
Iteration 13, loss = 1.35415688
Iteration 14, loss = 1.30591764
Iteration 15, loss = 1.26436817
Iteration 16, loss = 1.22562622
Iteration 17, loss = 1.18895881
Iteration 18, loss = 1.14766031
Iteration 19, loss = 1.11313602
Iteration 20, loss = 1.08611441
Iteration 21, loss = 1.04853023
Iteration 22, loss = 1.01944624
Iteration 23, loss = 0.99252434
Iteration 24, loss = 0.95907687
Iteration 25, loss = 0.93848565
Iteration 26, loss = 0.91030182
Iteration 27, loss = 0.88493463
Iteration 28, loss = 0.85838274
Iteration 29, loss = 0.83620231
Iteration 30, loss = 0.81740961
Iteration 31, loss = 0.78846589


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 4.73574726
Iteration 2, loss = 3.16894499
Iteration 3, loss = 2.35659998
Iteration 4, loss = 2.60501042
Iteration 5, loss = 1.63129359
Iteration 6, loss = 1.61827960
Iteration 7, loss = 1.34682007
Iteration 8, loss = 1.12425020
Iteration 9, loss = 1.00980119
Iteration 10, loss = 0.83666711
Iteration 11, loss = 0.80581203
Iteration 12, loss = 0.66588096
Iteration 13, loss = 0.62160855
Iteration 14, loss = 0.54992725
Iteration 15, loss = 0.48829379
Iteration 16, loss = 0.44432229
Iteration 17, loss = 0.39786257
Iteration 18, loss = 0.36691752
Iteration 19, loss = 0.33227892
Iteration 20, loss = 0.30422358
Iteration 21, loss = 0.27913710
Iteration 22, loss = 0.25754430
Iteration 23, loss = 0.23636702
Iteration 24, loss = 0.21841847
Iteration 25, loss = 0.20340914
Iteration 26, loss = 0.18828179
Iteration 27, loss = 0.17551226
Iteration 28, loss = 0.16322810
Iteration 29, loss = 0.15250940
Iteration 30, loss = 0.14314749
Iteration 31, loss = 0.13415411
Iteration 32, los